OOF CV: 0.015442

# NikoNeko NN

In [1]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import umap.umap_ as umap
from scipy.sparse.csgraph import connected_components
from sklearn.manifold import TSNE
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import VarianceThreshold

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 2000)

In [3]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [4]:
class Features:
    def __init__(self, train_features, valid_features, test_features, seed=42):
        self.train_features = train_features
        self.valid_features = valid_features
        self.test_features = test_features
        self.seed = seed
        self.GENES = [col for col in train_features.columns if col.startswith('g-')]
        self.CELLS = [col for col in train_features.columns if col.startswith('c-')]
        self.RANKGAUSS_GENES = []
        self.RANKGAUSS_CELLS = []
        
    def get_len_feature(self, feature_prefix):
        return len([col for col in self.train_features.columns if col.lower().startswith(feature_prefix)])
        
    def get_feature(self, feature_prefix):
        x_train = self.train_features[[col for col in self.train_features.columns if col.lower().startswith(feature_prefix)]].values
        x_valid = self.valid_features[[col for col in self.train_features.columns if col.lower().startswith(feature_prefix)]].values
        x_test = self.test_features[[col for col in self.train_features.columns if col.lower().startswith(feature_prefix)]].values
        # StandardScaler based on training data
        # do not apply for categorical features (cp_)
        if STANDARDSCALER == True and feature_prefix != 'cp_':
            scaler = StandardScaler()
            scaler.fit(x_train)
            x_train = scaler.transform(x_train)
            x_valid = scaler.transform(x_valid)
            x_test = scaler.transform(x_test)
        return x_train, x_valid, x_test

    def get_target(self):
        y_train = self.train_features[TARGET_COLS].values.astype(np.float32)
        y_valid = self.valid_features[TARGET_COLS].values.astype(np.float32)
        return y_train, y_valid

    def _get_dummies(self):
        data = pd.concat([self.train_features, self.valid_features, self.test_features], axis=0)
        data = pd.get_dummies(data, columns=['cp_time','cp_dose'], prefix='cp_')
        self.train_features = data[ : self.train_features.shape[0]]
        self.valid_features = data[self.train_features.shape[0] : -self.test_features.shape[0]]
        self.test_features = data[-self.test_features.shape[0] : ]

    def _rankgauss(self, cols, col_type):
        # apply rankgauss with each column of GENES, CELLS groups
        for col in cols:
            if col_type == 'GENES':
                self.RANKGAUSS_GENES.append(f'rankgauss_{col}')
            if col_type == 'CELLS':
                self.RANKGAUSS_CELLS.append(f'rankgauss_{col}')
            transformer = QuantileTransformer(n_quantiles=100, random_state=self.seed, output_distribution="normal")
            vec_len = len(self.train_features[col].values)
            vec_len_valid = len(self.valid_features[col].values)
            vec_len_test = len(self.test_features[col].values)
            raw_vec = self.train_features[col].values.reshape(vec_len, 1)
            transformer.fit(raw_vec)
            self.train_features[f'rankgauss_{col}'] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
            self.valid_features[f'rankgauss_{col}'] = transformer.transform(self.valid_features[col].values.reshape(vec_len_valid, 1)).reshape(1, vec_len_valid)[0]
            self.test_features[f'rankgauss_{col}'] = transformer.transform(self.test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]
    
    def _truncatedSVD(self, cols, n_comp, col_type) :
        # pca = (PCA(n_components=n_comp, random_state=self.seed).fit(train_features[cols]))
        svd = (TruncatedSVD(n_components=n_comp, n_iter=10, random_state=self.seed).fit(self.train_features[cols]))
        train2 = svd.transform(self.train_features[cols])
        valid2 = svd.transform(self.valid_features[cols])
        test2 = svd.transform(self.test_features[cols])
        
        if col_type == 'GENES':
            train2 = pd.DataFrame(train2, columns=[f'svd_g-{i}' for i in range(n_comp)])
            valid2 = pd.DataFrame(valid2, columns=[f'svd_g-{i}' for i in range(n_comp)])
            test2 = pd.DataFrame(test2, columns=[f'svd_g-{i}' for i in range(n_comp)])
        if col_type == 'CELLS':
            train2 = pd.DataFrame(train2, columns=[f'svd_c-{i}' for i in range(n_comp)])
            valid2 = pd.DataFrame(valid2, columns=[f'svd_c-{i}' for i in range(n_comp)])
            test2 = pd.DataFrame(test2, columns=[f'svd_c-{i}' for i in range(n_comp)])
        if col_type == 'RANKGAUSS_GENES':
            train2 = pd.DataFrame(train2, columns=[f'svd_rankgauss_g-{i}' for i in range(n_comp)])
            valid2 = pd.DataFrame(valid2, columns=[f'svd_rankgauss_g-{i}' for i in range(n_comp)])
            test2 = pd.DataFrame(test2, columns=[f'svd_rankgauss_g-{i}' for i in range(n_comp)])
        if col_type == 'RANKGAUSS_CELLS':
            train2 = pd.DataFrame(train2, columns=[f'svd_rankgauss_c-{i}' for i in range(n_comp)])
            valid2 = pd.DataFrame(valid2, columns=[f'svd_rankgauss_c-{i}' for i in range(n_comp)])
            test2 = pd.DataFrame(test2, columns=[f'svd_rankgauss_c-{i}' for i in range(n_comp)])
    
        self.train_features = pd.concat((self.train_features, train2), axis=1)
        self.valid_features = pd.concat((self.valid_features, valid2), axis=1)
        self.test_features = pd.concat((self.test_features, test2), axis=1)
        
    def _variance_threshold_selector(self, data_train, data_valid, data_test, threshold=0.5):
        # fit only train data, not fit for concat[train, valid, test]
        selector = VarianceThreshold(threshold)
        selector.fit(data_train)
        train2 = data_train[data_train.columns[selector.get_support(indices=True)]]
        valid2 = data_valid[data_train.columns[selector.get_support(indices=True)]]
        test2 = data_test[data_train.columns[selector.get_support(indices=True)]]
        return train2, valid2, test2
    
    def _make_features_selection(self, threshold=0.5):
        # Remove Target, kfold columns for features selection
        feature_cols = [c for c in self.train_features.columns if c not in TARGET_COLS]
        feature_cols = [c for c in feature_cols if c not in ['kfold']]
        # data = pd.concat([self.train_features[feature_cols], self.valid_features[feature_cols], self.test_features[feature_cols]], axis=0)
        # Remove 'sig_id', 'cp_time', 'cp_dose' for features selection 
        # Since cp_type was dropped, sig_id(column_0), cp_time(column_1), cp_dose(column_2)
        # data_transformed = self._variance_threshold_selector(data.iloc[:, 3:], threshold=threshold)
        # train_features_transformed = data_transformed[ : self.train_features.shape[0]]
        # valid_features_transformed = data_transformed[self.train_features.shape[0] : -self.test_features.shape[0]]
        # test_features_transformed = data_transformed[-self.test_features.shape[0] : ]
        data_train = self.train_features[feature_cols]
        data_valid = self.valid_features[feature_cols]
        data_test = self.test_features[feature_cols]
        train_features_transformed, valid_features_transformed, test_features_transformed \
        = self._variance_threshold_selector(data_train.iloc[:, 3:], data_valid.iloc[:, 3:], data_test.iloc[:, 3:], threshold=threshold)

        # 210 = 4 columns('sig_id','cp_time','cp_dose', 'kfold') + 206 columns(TARGET_COLS)
        self.train_features = pd.DataFrame(self.train_features[['sig_id','cp_time','cp_dose','kfold'] + TARGET_COLS].values.reshape(-1, 210),\
                                      columns=['sig_id','cp_time','cp_dose','kfold'] + TARGET_COLS)
        self.train_features = pd.concat([self.train_features, pd.DataFrame(train_features_transformed)], axis=1)
        self.valid_features = pd.DataFrame(self.valid_features[['sig_id','cp_time','cp_dose','kfold'] + TARGET_COLS].values.reshape(-1, 210),\
                                     columns=['sig_id','cp_time','cp_dose','kfold'] + TARGET_COLS)
        self.valid_features = pd.concat([self.valid_features, pd.DataFrame(valid_features_transformed)], axis=1)
        self.test_features = pd.DataFrame(self.test_features[['sig_id','cp_time','cp_dose']].values.reshape(-1, 3),\
                                     columns=['sig_id','cp_time','cp_dose'])
        self.test_features = pd.concat([self.test_features, pd.DataFrame(test_features_transformed)], axis=1)        
        
    def _get_dummies(self):
        data = pd.concat([self.train_features, self.valid_features, self.test_features], axis=0)
        data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
        self.train_features = data[ : self.train_features.shape[0]]
        self.valid_features = data[self.train_features.shape[0] : -self.test_features.shape[0]]
        self.test_features = data[-self.test_features.shape[0] : ]
                
    def make_features(self) :
        # rankgauss features (rankgauss_{col})
        self._rankgauss(self.GENES, "GENES")
        self._rankgauss(self.CELLS, "CELLS")

        # TruncatedSVD features (svd_)
        self._truncatedSVD(self.GENES, 50, "GENES")
        self._truncatedSVD(self.CELLS, 50, "CELLS")
        self._truncatedSVD(self.RANKGAUSS_GENES, 50, "RANKGAUSS_GENES")
        self._truncatedSVD(self.RANKGAUSS_CELLS, 50, "RANKGAUSS_CELLS")

        # feature Selection using Variance Encoding
        self._make_features_selection(threshold=0.5)
        
        # get_dummies for 'cp_time','cp_dose'
        self._get_dummies()
        
        return self.train_features, self.valid_features, self.test_features


In [5]:
class MoADataset:
    def __init__(self, g_feats, c_feats, rg_g_feats, rg_c_feats, svd_g_feats, svd_c_feats, rg_g_rg_c_feats, targets):
    # def __init__(self, g_feats, c_feats, rg_g_feats, rg_c_feats, svd_g_feats, svd_c_feats,
    #             g_c_feats, rg_g_rg_c_feats, svd_g_svd_c_feats, svd_rg_g_svd_rg_c_feats, targets):
        self.g_features = g_feats
        self.c_features = c_feats
        self.rg_g_features = rg_g_feats
        self.rg_c_features = rg_c_feats
        self.svd_g_features = svd_g_feats
        self.svd_c_features = svd_c_feats
        self.rg_g_rg_c_features = rg_g_rg_c_feats
        self.targets = targets
        
    def __len__(self):
        return (self.g_features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x_g' : torch.tensor(self.g_features[idx, :], dtype=torch.float),
            'x_c' : torch.tensor(self.c_features[idx, :], dtype=torch.float),
            'x_rg_g' : torch.tensor(self.rg_g_features[idx, :], dtype=torch.float),
            'x_rg_c' : torch.tensor(self.rg_c_features[idx, :], dtype=torch.float),
            'x_svd_g' : torch.tensor(self.svd_g_features[idx, :], dtype=torch.float),
            'x_svd_c' : torch.tensor(self.svd_c_features[idx, :], dtype=torch.float),
            'x_rg_g_rg_c' : torch.tensor(self.rg_g_rg_c_features[idx, :], dtype=torch.float),
            # 'x_g_c' : torch.tensor(self.g_c_features[idx, :], dtype=torch.float),
            # 'x_svd_g_svd_c' : torch.tensor(self.svd_g_svd_c_features[idx, :], dtype=torch.float),
            # 'x_svd_rg_g_svd_rg_c' : torch.tensor(self.svd_rg_g_svd_rg_c_features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, g_feats, c_feats, rg_g_feats, rg_c_feats, svd_g_feats, svd_c_feats, rg_g_rg_c_feats):
    # def __init__(self, g_feats, c_feats, rg_g_feats, rg_c_feats, svd_g_feats, svd_c_feats,
    #             g_c_feats, rg_g_rg_c_feats, svd_g_svd_c_feats, svd_rg_g_svd_rg_c_feats):
        self.g_features = g_feats
        self.c_features = c_feats
        self.rg_g_features = rg_g_feats
        self.rg_c_features = rg_c_feats
        self.svd_g_features = svd_g_feats
        self.svd_c_features = svd_c_feats
        self.rg_g_rg_c_features = rg_g_rg_c_feats
        
    def __len__(self):
        return (self.g_features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x_g' : torch.tensor(self.g_features[idx, :], dtype=torch.float),
            'x_c' : torch.tensor(self.c_features[idx, :], dtype=torch.float),
            'x_rg_g' : torch.tensor(self.rg_g_features[idx, :], dtype=torch.float),
            'x_rg_c' : torch.tensor(self.rg_c_features[idx, :], dtype=torch.float),
            'x_svd_g' : torch.tensor(self.svd_g_features[idx, :], dtype=torch.float),
            'x_svd_c' : torch.tensor(self.svd_c_features[idx, :], dtype=torch.float),
            'x_rg_g_rg_c' : torch.tensor(self.rg_g_rg_c_features[idx, :], dtype=torch.float)
            # 'x_g_c' : torch.tensor(self.g_c_features[idx, :], dtype=torch.float),
            # 'x_svd_g_svd_c' : torch.tensor(self.svd_g_svd_c_features[idx, :], dtype=torch.float),
            # 'x_svd_rg_g_svd_rg_c' : torch.tensor(self.svd_rg_g_svd_rg_c_features[idx, :], dtype=torch.float)
        }
        return dct

In [6]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [7]:
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)
    
class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(F.softplus(x))

class Model(nn.Module):
    def __init__(self, num_g_feats, num_c_feats, num_rg_g_feats, num_rg_c_feats,
                 num_svd_g_feats, num_svd_c_feats, num_rg_g_rg_c_feats):
    # def __init__(self, num_g_feats, num_c_feats, num_rg_g_feats, num_rg_c_feats, num_svd_g_feats, num_svd_c_feats,
    #             num_g_c_feats, num_rg_g_rg_c_feats, num_svd_g_svd_c_feats, num_svd_rg_g_svd_rg_c_feats):
        super(Model, self).__init__()
        self.batch_norm_g = nn.BatchNorm1d(num_g_feats)
        self.dropout_g = nn.Dropout(0.2)
        self.prelu_g = nn.PReLU()
        self.linear_g = nn.Linear(num_g_feats, HIDDEN_SIZE_G)
        self.dense_g = nn.utils.weight_norm(self.linear_g)

        self.batch_norm_c = nn.BatchNorm1d(num_c_feats)
        self.dropout_c = nn.Dropout(0.2)
        self.prelu_c = nn.PReLU()
        self.linear_c = nn.Linear(num_c_feats, HIDDEN_SIZE_C)
        self.dense_c = nn.utils.weight_norm(self.linear_c)

        self.batch_norm_rg_g = nn.BatchNorm1d(num_rg_g_feats)
        self.dropout_rg_g = nn.Dropout(0.2)
        self.prelu_rg_g = nn.PReLU()
        self.linear_rg_g = nn.Linear(num_rg_g_feats, HIDDEN_SIZE_RG_G)
        self.dense_rg_g = nn.utils.weight_norm(self.linear_rg_g)

        self.batch_norm_rg_c = nn.BatchNorm1d(num_rg_c_feats)
        self.dropout_rg_c = nn.Dropout(0.2)
        self.prelu_rg_c = nn.PReLU()
        self.linear_rg_c = nn.Linear(num_rg_c_feats, HIDDEN_SIZE_RG_C)
        self.dense_rg_c = nn.utils.weight_norm(self.linear_rg_c)

        self.batch_norm_svd_g = nn.BatchNorm1d(num_svd_g_feats)
        self.dropout_svd_g = nn.Dropout(0.2)
        self.prelu_svd_g = nn.PReLU()
        self.linear_svd_g = nn.Linear(num_svd_g_feats, HIDDEN_SIZE_SVD_G)
        self.dense_svd_g = nn.utils.weight_norm(self.linear_svd_g)

        self.batch_norm_svd_c = nn.BatchNorm1d(num_svd_c_feats)
        self.dropout_svd_c = nn.Dropout(0.2)
        self.prelu_svd_c = nn.PReLU()
        self.linear_svd_c = nn.Linear(num_svd_c_feats, HIDDEN_SIZE_SVD_C)
        self.dense_svd_c = nn.utils.weight_norm(self.linear_svd_c)
        
        # self.batch_norm_g_c = nn.BatchNorm1d(num_g_c_feats)
        # self.dropout_g_c = nn.Dropout(0.2)
        # self.prelu_g_c = nn.PReLU()
        # self.linear_g_c = nn.Linear(num_g_c_feats, HIDDEN_SIZE_G_C)
        # self.dense_g_c = nn.utils.weight_norm(self.linear_g_c)

        self.batch_norm_rg_g_rg_c = nn.BatchNorm1d(num_rg_g_rg_c_feats)
        self.dropout_rg_g_rg_c = nn.Dropout(0.2)
        self.prelu_rg_g_rg_c = nn.PReLU()
        self.linear_rg_g_rg_c = nn.Linear(num_rg_g_rg_c_feats, HIDDEN_SIZE_RG_G_RG_C)
        self.dense_rg_g_rg_c = nn.utils.weight_norm(self.linear_rg_g_rg_c)
        
        # self.batch_norm_svd_g_svd_c = nn.BatchNorm1d(num_svd_g_svd_c_feats)
        # self.dropout_svd_g_svd_c = nn.Dropout(0.2)
        # self.prelu_svd_g_svd_c = nn.PReLU()
        # self.linear_svd_g_svd_c = nn.Linear(num_svd_g_svd_c_feats, HIDDEN_SIZE_SVD_G_SVD_C)
        # self.dense_svd_g_svd_c = nn.utils.weight_norm(self.linear_svd_g_svd_c)

        # self.batch_norm_svd_rg_g_svd_rg_c = nn.BatchNorm1d(num_svd_rg_g_svd_rg_c_feats)
        # self.dropout_svd_rg_g_svd_rg_c = nn.Dropout(0.2)
        # self.prelu_svd_rg_g_svd_rg_c = nn.PReLU()
        # self.linear_svd_rg_g_svd_rg_c = nn.Linear(num_svd_rg_g_svd_rg_c_feats, HIDDEN_SIZE_SVD_RG_G_SVD_RG_C)
        # self.dense_svd_rg_g_svd_rg_c = nn.utils.weight_norm(self.linear_svd_rg_g_svd_rg_c)
        
        # self.batch_norm2 = nn.BatchNorm1d(HIDDEN_SIZE_G + HIDDEN_SIZE_C \
        #                                  + HIDDEN_SIZE_RG_G + HIDDEN_SIZE_RG_C + HIDDEN_SIZE_SVD_G + HIDDEN_SIZE_SVD_C \
        #                                  + HIDDEN_SIZE_G_C + HIDDEN_SIZE_RG_G_RG_C + HIDDEN_SIZE_SVD_G_SVD_C + HIDDEN_SIZE_SVD_RG_G_SVD_RG_C)
        self.batch_norm2 = nn.BatchNorm1d(HIDDEN_SIZE_G + HIDDEN_SIZE_C \
                                          + HIDDEN_SIZE_RG_G + HIDDEN_SIZE_RG_C + HIDDEN_SIZE_SVD_G + HIDDEN_SIZE_SVD_C \
                                          + HIDDEN_SIZE_RG_G_RG_C)
        self.dropout2 = nn.Dropout(0.4)
        self.prelu2 = nn.PReLU()
        # self.linear2 = nn.Linear(HIDDEN_SIZE_G + HIDDEN_SIZE_C \
        #                         + HIDDEN_SIZE_RG_G + HIDDEN_SIZE_RG_C + HIDDEN_SIZE_SVD_G + HIDDEN_SIZE_SVD_C \
        #                         + HIDDEN_SIZE_G_C + HIDDEN_SIZE_RG_G_RG_C + HIDDEN_SIZE_SVD_G_SVD_C + HIDDEN_SIZE_SVD_RG_G_SVD_RG_C, HIDDEN_SIZE)
        self.linear2 = nn.Linear(HIDDEN_SIZE_G + HIDDEN_SIZE_C \
                                 + HIDDEN_SIZE_RG_G + HIDDEN_SIZE_RG_C + HIDDEN_SIZE_SVD_G + HIDDEN_SIZE_SVD_C \
                                 + HIDDEN_SIZE_RG_G_RG_C, HIDDEN_SIZE)
        self.dense2 = nn.utils.weight_norm(self.linear2)

        self.batch_norm3 = nn.BatchNorm1d(HIDDEN_SIZE)
        self.dropout3 = nn.Dropout(0.4)
        self.prelu3 = nn.PReLU()
        self.linear3 = nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE)
        self.dense3 = nn.utils.weight_norm(self.linear3)

        self.batch_norm4 = nn.BatchNorm1d(HIDDEN_SIZE)
        self.dropout4 = nn.Dropout(0.4)
        self.prelu4 = nn.PReLU()
        self.linear4 = nn.Linear(HIDDEN_SIZE, HIDDEN_SIZE)
        self.dense4 = nn.utils.weight_norm(self.linear4)
                
        self.batch_norm99 = nn.BatchNorm1d(HIDDEN_SIZE)
        self.dropout99 = nn.Dropout(0.4)
        self.linear99 = nn.Linear(HIDDEN_SIZE, NUM_TARGETS)
        self.dense99 = nn.utils.weight_norm(self.linear99)
        self.sigmoid99 = nn.Sigmoid()
        
    def recalibrate_layer(self, layer):
        if(torch.isnan(layer.weight_v).sum() > 0):
            print ('recalibrate layer.weight_v')
            layer.weight_v = torch.nn.Parameter(torch.where(torch.isnan(layer.weight_v), torch.zeros_like(layer.weight_v), layer.weight_v))
            layer.weight_v = torch.nn.Parameter(layer.weight_v + 1e-7)

        if(torch.isnan(layer.weight).sum() > 0):
            print ('recalibrate layer.weight')
            layer.weight = torch.where(torch.isnan(layer.weight), torch.zeros_like(layer.weight), layer.weight)
            layer.weight += 1e-7

    def forward(self, x_g, x_c, x_rg_g, x_rg_c, x_svd_g, x_svd_c, x_rg_g_rg_c):
    # def forward(self, x_g, x_c, x_rg_g, x_rg_c, x_svd_g, x_svd_c, x_g_c, x_rg_g_rg_c, x_svd_g_svd_c, x_svd_rg_g_svd_rg_c):
        x_g = self.batch_norm_g(x_g)
        x_g = self.dropout_g(x_g)
        self.recalibrate_layer(self.dense_g)
        x_g = self.prelu_g(self.dense_g(x_g))

        x_c = self.batch_norm_c(x_c)
        x_c = self.dropout_c(x_c)
        self.recalibrate_layer(self.dense_c)
        x_c = self.prelu_c(self.dense_c(x_c))

        x_rg_g = self.batch_norm_rg_g(x_rg_g)
        x_rg_g = self.dropout_rg_g(x_rg_g)
        self.recalibrate_layer(self.dense_rg_g)
        x_rg_g = self.prelu_rg_g(self.dense_rg_g(x_rg_g))

        x_rg_c = self.batch_norm_rg_c(x_rg_c)
        x_rg_c = self.dropout_rg_c(x_rg_c)
        self.recalibrate_layer(self.dense_rg_c)
        x_rg_c = self.prelu_rg_c(self.dense_rg_c(x_rg_c))

        x_svd_g = self.batch_norm_svd_g(x_svd_g)
        x_svd_g = self.dropout_svd_g(x_svd_g)
        self.recalibrate_layer(self.dense_svd_g)
        x_svd_g = self.prelu_svd_g(self.dense_svd_g(x_svd_g))

        x_svd_c = self.batch_norm_svd_c(x_svd_c)
        x_svd_c = self.dropout_svd_c(x_svd_c)
        self.recalibrate_layer(self.dense_svd_c)
        x_svd_c = self.prelu_svd_c(self.dense_svd_c(x_svd_c))

        # x_g_c = self.batch_norm_g_c(x_g_c)
        # x_g_c = self.dropout_g_c(x_g_c)
        # self.recalibrate_layer(self.dense_g_c)
        # x_g_c = self.prelu_g_c(self.dense_g_c(x_g_c))

        x_rg_g_rg_c = self.batch_norm_rg_g_rg_c(x_rg_g_rg_c)
        x_rg_g_rg_c = self.dropout_rg_g_rg_c(x_rg_g_rg_c)
        self.recalibrate_layer(self.dense_rg_g_rg_c)
        x_rg_g_rg_c = self.prelu_rg_g_rg_c(self.dense_rg_g_rg_c(x_rg_g_rg_c))
        
        # x_svd_g_svd_c = self.batch_norm_svd_g_svd_c(x_svd_g_svd_c)
        # x_svd_g_svd_c = self.dropout_svd_g_svd_c(x_svd_g_svd_c)
        # self.recalibrate_layer(self.dense_svd_g_svd_c)
        # x_svd_g_svd_c = self.prelu_svd_g_svd_c(self.dense_svd_g_svd_c(x_svd_g_svd_c))

        # x_svd_rg_g_svd_rg_c = self.batch_norm_svd_rg_g_svd_rg_c(x_svd_rg_g_svd_rg_c)
        # x_svd_rg_g_svd_rg_c = self.dropout_svd_rg_g_svd_rg_c(x_svd_rg_g_svd_rg_c)
        # self.recalibrate_layer(self.dense_svd_rg_g_svd_rg_c)
        # x_svd_rg_g_svd_rg_c = self.prelu_svd_rg_g_svd_rg_c(self.dense_svd_rg_g_svd_rg_c(x_svd_rg_g_svd_rg_c))
             
        # x = torch.cat([x_g, x_c, x_rg_g, x_rg_c, x_svd_g, x_svd_c, x_g_c, x_rg_g_rg_c, x_svd_g_svd_c, x_svd_rg_g_svd_rg_c], axis=1)
        x = torch.cat([x_g, x_c, x_rg_g, x_rg_c, x_svd_g, x_svd_c, x_rg_g_rg_c], axis=1)
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        self.recalibrate_layer(self.dense2)
        x = self.prelu2(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        self.recalibrate_layer(self.dense3)
        x = self.prelu3(self.dense3(x))

        # x = self.batch_norm4(x)
        # x = self.dropout4(x)
        # self.recalibrate_layer(self.dense4)
        # x = self.prelu4(self.dense4(x))

        x = self.batch_norm99(x)
        x = self.dropout99(x)
        self.recalibrate_layer(self.dense99)
        x = self.dense99(x)
    
        return x

In [8]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device, seed, fold):
    model.train()
    final_loss = 0
    
    if MIXUP == True and MIXUP_SEED_FOLD[seed, fold] == 1:
        print('Using MixUp')

    for data in dataloader:
        optimizer.zero_grad()
        inputs_g = data['x_g'].to(device)
        inputs_c = data['x_c'].to(device)
        inputs_rg_g = data['x_rg_g'].to(device)
        inputs_rg_c = data['x_rg_c'].to(device)
        inputs_svd_g = data['x_svd_g'].to(device)
        inputs_svd_c = data['x_svd_c'].to(device)
        # inputs_g_c = data['x_g_c'].to(device)
        inputs_rg_g_rg_c = data['x_rg_g_rg_c'].to(device)
        # inputs_svd_g_svd_c = data['x_svd_g_svd_c'].to(device)
        # inputs_svd_rg_g_svd_rg_c = data['x_svd_rg_g_svd_rg_c'].to(device)
        targets = data['y'].to(device)

        if MIXUP == True and MIXUP_SEED_FOLD[seed, fold] == 1:
            # alpha = 0.2
            # alpha = 0.05
            # alpha = 0.2
            # alpha = 0.01
            size = len(targets)
            g = np.random.beta(MIXUP_ALPHA, MIXUP_ALPHA)
            index = torch.randperm(size, dtype=torch.long)
            inputs_g = (1-g) * inputs_g[index] + g*inputs_g
            inputs_c = (1-g) * inputs_c[index] + g*inputs_c
            inputs_rg_g = (1-g) * inputs_rg_g[index] + g*inputs_rg_g
            inputs_rg_c = (1-g) * inputs_rg_c[index] + g*inputs_rg_c
            inputs_svd_g = (1-g) * inputs_svd_g[index] + g*inputs_svd_g
            inputs_svd_c = (1-g) * inputs_svd_c[index] + g*inputs_svd_c
            inputs_rg_g_rg_c = (1-g) * inputs_rg_g_rg_c[index] + g*inputs_rg_g_rg_c
            targets = (1-g) * targets[index] + g*targets

        outputs = model(inputs_g, inputs_c, inputs_rg_g, inputs_rg_c,
                        inputs_svd_g, inputs_svd_c, inputs_rg_g_rg_c)
        # outputs = model(inputs_g, inputs_c,
        #                inputs_rg_g, inputs_rg_c,
        #                inputs_svd_g, inputs_svd_c,
        #                inputs_g_c, inputs_rg_g_rg_c, inputs_svd_g_svd_c, inputs_svd_rg_g_svd_rg_c)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs_g = data['x_g'].to(device)
        inputs_c = data['x_c'].to(device)
        inputs_rg_g = data['x_rg_g'].to(device)
        inputs_rg_c = data['x_rg_c'].to(device)
        inputs_svd_g = data['x_svd_g'].to(device)
        inputs_svd_c = data['x_svd_c'].to(device)
        inputs_rg_g_rg_c = data['x_rg_g_rg_c'].to(device)
        # inputs_g_c = data['x_g_c'].to(device)  
        # inputs_svd_g_svd_c = data['x_svd_g_svd_c'].to(device)  
        # inputs_svd_rg_g_svd_rg_c = data['x_svd_rg_g_svd_rg_c'].to(device)
        targets = data['y'].to(device)

        outputs = model(inputs_g, inputs_c, inputs_rg_g, inputs_rg_c,
                        inputs_svd_g, inputs_svd_c, inputs_rg_g_rg_c)
        # outputs = model(inputs_g, inputs_c,
        #                inputs_rg_g, inputs_rg_c,
        #                inputs_svd_g, inputs_svd_c,
        #                inputs_g_c, inputs_rg_g_rg_c, inputs_svd_g_svd_c, inputs_svd_rg_g_svd_rg_c)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs_g = data['x_g'].to(device)
        inputs_c = data['x_c'].to(device)
        inputs_rg_g = data['x_rg_g'].to(device)
        inputs_rg_c = data['x_rg_c'].to(device)
        inputs_svd_g = data['x_svd_g'].to(device)
        inputs_svd_c = data['x_svd_c'].to(device)
        inputs_rg_g_rg_c = data['x_rg_g_rg_c'].to(device)
        # inputs_g_c = data['x_g_c'].to(device)
        # inputs_svd_g_svd_c = data['x_svd_g_svd_c'].to(device)  
        # inputs_svd_rg_g_svd_rg_c = data['x_svd_rg_g_svd_rg_c'].to(device)

        with torch.no_grad():
            outputs = model(inputs_g, inputs_c, inputs_rg_g, inputs_rg_c,
                            inputs_svd_g, inputs_svd_c, inputs_rg_g_rg_c)
            # outputs = model(inputs_g, inputs_c,
            #                inputs_rg_g, inputs_rg_c,
            #                inputs_svd_g, inputs_svd_c,
            #                inputs_g_c, inputs_rg_g_rg_c, inputs_svd_g_svd_c, inputs_svd_rg_g_svd_rg_c)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

In [9]:
def run_k_fold(folds, NFOLDS, seed, model_type):
    oof = np.zeros((len(train), len(TARGET_COLS)))
    predictions = np.zeros((len(test), len(TARGET_COLS)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(folds, fold, seed, model_type)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

def run_training(folds, fold, seed, model_type):
    
    seed_everything(seed)
    
    train = folds
    test_ = test
    
    trn_idx = train[train['kfold'] != fold].index
    val_idx = train[train['kfold'] == fold].index
    
    train_df = train[train['kfold'] != fold].reset_index(drop=True)
    valid_df = train[train['kfold'] == fold].reset_index(drop=True)

    # Feature engineering
    features = Features(train_df, valid_df, test_, seed=seed)
    train_df, valid_df, test_ = features.make_features()

    #--------------------- TRAINING ---------------------
    x_cp_train, x_cp_valid, x_cp_test = features.get_feature('cp_')
    x_g_train, x_g_valid, x_g_test = features.get_feature('g-')
    x_c_train, x_c_valid, x_c_test = features.get_feature('c-')
    x_rg_g_train, x_rg_g_valid, x_rg_g_test = features.get_feature('rankgauss_g-')
    x_rg_c_train, x_rg_c_valid, x_rg_c_test = features.get_feature('rankgauss_c-')
    x_svd_g_train, x_svd_g_valid, x_svd_g_test = features.get_feature('svd_g-')
    x_svd_c_train, x_svd_c_valid, x_svd_c_test = features.get_feature('svd_c-')
    x_svd_rg_g_train, x_svd_rg_g_valid, x_svd_rg_g_test = features.get_feature('svd_rankgauss_g-')
    x_svd_rg_c_train, x_svd_rg_c_valid, x_svd_rg_c_test = features.get_feature('svd_rankgauss_c-')
    y_train, y_valid = features.get_target()
    # print(x_g_train.shape, x_g_valid.shape, x_g_test.shape)

    x_g_train_cat = np.concatenate((x_g_train, x_svd_g_train, x_cp_train), axis=1)
    x_c_train_cat = np.concatenate((x_c_train, x_svd_c_train, x_cp_train), axis=1)
    x_g_valid_cat = np.concatenate((x_g_valid, x_svd_g_valid, x_cp_valid), axis=1)
    x_c_valid_cat = np.concatenate((x_c_valid, x_svd_c_valid, x_cp_valid), axis=1)
    x_rg_g_train_cat = np.concatenate((x_rg_g_train, x_svd_rg_g_train, x_cp_train), axis=1)
    x_rg_c_train_cat = np.concatenate((x_rg_c_train, x_svd_rg_c_train, x_cp_train), axis=1)
    x_rg_g_valid_cat = np.concatenate((x_rg_g_valid, x_svd_rg_g_valid, x_cp_valid), axis=1)
    x_rg_c_valid_cat = np.concatenate((x_rg_c_valid, x_svd_rg_c_valid, x_cp_valid), axis=1)     
    x_svd_g_train_cat = np.concatenate((x_svd_g_train, x_svd_rg_g_train), axis=1)
    x_svd_c_train_cat = np.concatenate((x_svd_c_train, x_svd_rg_c_train), axis=1)
    x_svd_g_valid_cat = np.concatenate((x_svd_g_valid, x_svd_rg_g_valid), axis=1)
    x_svd_c_valid_cat = np.concatenate((x_svd_c_valid, x_svd_rg_c_valid), axis=1)
    x_rg_g_rg_c_train_cat = np.concatenate((x_rg_g_train, x_rg_c_train, x_cp_train), axis=1)
    x_rg_g_rg_c_valid_cat = np.concatenate((x_rg_g_valid, x_rg_c_valid, x_cp_valid), axis=1)
    # x_g_c_train_cat = np.concatenate((x_g_train, x_c_train, x_cp_train), axis=1)
    # x_g_c_valid_cat = np.concatenate((x_g_valid, x_c_valid, x_cp_valid), axis=1)
    # x_svd_g_svd_c_train_cat = np.concatenate((x_svd_g_train, x_svd_c_train), axis=1)
    # x_svd_g_svd_c_valid_cat = np.concatenate((x_svd_g_valid, x_svd_c_valid), axis=1)
    # x_svd_rg_g_svd_rg_c_train_cat = np.concatenate((x_svd_rg_g_train, x_svd_rg_c_train), axis=1)
    # x_svd_rg_g_svd_rg_c_valid_cat = np.concatenate((x_svd_rg_g_valid, x_svd_rg_c_valid), axis=1)

    train_dataset = MoADataset(x_g_train_cat, x_c_train_cat, x_rg_g_train_cat, x_rg_c_train_cat,
                               x_svd_g_train_cat, x_svd_c_train_cat, x_rg_g_rg_c_train_cat, y_train)
    valid_dataset = MoADataset(x_g_valid_cat, x_c_valid_cat, x_rg_g_valid_cat, x_rg_c_valid_cat,
                               x_svd_g_valid_cat, x_svd_c_valid_cat, x_rg_g_rg_c_valid_cat,  y_valid) 
    # train_dataset = MoADataset(x_g_train_cat, x_c_train_cat,
    #                           x_rg_g_train_cat, x_rg_c_train_cat,
    #                           x_svd_g_train_cat, x_svd_c_train_cat,
    #                           x_g_c_train_cat, x_rg_g_rg_c_train_cat, x_svd_g_svd_c_train_cat, x_svd_rg_g_svd_rg_c_train_cat,
    #                           y_train)
    # valid_dataset = MoADataset(x_g_valid_cat, x_c_valid_cat,
    #                           x_rg_g_valid_cat, x_rg_c_valid_cat,
    #                           x_svd_g_valid_cat, x_svd_c_valid_cat,
    #                           x_g_c_valid_cat, x_rg_g_rg_c_valid_cat, x_svd_g_svd_c_valid_cat, x_svd_rg_g_svd_rg_c_valid_cat,
    #                           y_valid)    
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    num_cpfeats = features.get_len_feature('cp_')
    num_gfeats = features.get_len_feature('g-')
    num_cfeats = features.get_len_feature('c-')
    num_rg_gfeats = features.get_len_feature('rankgauss_g-')
    num_rg_cfeats = features.get_len_feature('rankgauss_c-')
    num_svd_gfeats = features.get_len_feature('svd_g-')
    num_svd_cfeats = features.get_len_feature('svd_c-')
    num_svd_rg_gfeats = features.get_len_feature('svd_rankgauss_g-')
    num_svd_rg_cfeats = features.get_len_feature('svd_rankgauss_c-')
    print(f"cpfeats : {num_cpfeats}")
    print(f"gfeats : {num_gfeats}")
    print(f"cfeats : {num_cfeats}")
    print(f"rg_gfeats : {num_rg_gfeats}")
    print(f"rg_cfeats : {num_rg_cfeats}")
    print(f"svd_gfeats : {num_svd_gfeats}")
    print(f"svd_cfeats : {num_svd_cfeats} ")
    print(f"svd_rg_gfeats : {num_svd_rg_gfeats}")
    print(f"svd_rg_cfeats : {num_svd_rg_cfeats}")
    
    model = Model(
        num_g_feats=num_gfeats + num_svd_gfeats + num_cpfeats,
        num_c_feats=num_cfeats + num_svd_cfeats + num_cpfeats,
        num_rg_g_feats=num_rg_gfeats + num_svd_rg_gfeats + num_cpfeats,
        num_rg_c_feats=num_rg_cfeats + num_svd_rg_cfeats + num_cpfeats,
        num_svd_g_feats=num_svd_gfeats + num_svd_rg_gfeats,
        num_svd_c_feats=num_svd_cfeats + num_svd_rg_cfeats,
        num_rg_g_rg_c_feats=num_rg_gfeats + num_rg_cfeats + num_cpfeats
        # num_g_c_feats=num_gfeats + num_cfeats + num_cpfeats,
        # num_svd_g_svd_c_feats=num_svd_gfeats + num_svd_cfeats,
        # num_svd_rg_g_svd_rg_c_feats=num_svd_rg_gfeats + num_svd_rg_cfeats
    )
    
    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    # optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                             max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    # scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer,T_max=len(trainloader), eta_min=0)
    # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min')
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
    
    oof = np.zeros((len(train), target.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    if INFERENCE == False :
        for epoch in range(EPOCHS):
            
            if LABEL_SMOOTHING == True:
                train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE, seed, fold)
            if LABEL_SMOOTHING == False:
                train_loss = train_fn(model, optimizer,scheduler, loss_fn, trainloader, DEVICE, seed, fold)
            valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
            
            if valid_loss < best_loss:
                best_loss = valid_loss
                oof[val_idx] = valid_preds
                if model_type == 'normal':
                    torch.save(model.state_dict(), f"NORMAL_SEED{seed}_FOLD{fold}.pth")
                if model_type == 'chris':
                    torch.save(model.state_dict(), f"CHRIS_SEED{seed}_FOLD{fold}.pth")
                if model_type == 'fkubota':
                    torch.save(model.state_dict(), f"FKUBOTA_SEED{seed}_FOLD{fold}.pth")

            elif(EARLY_STOP == True):
                early_step += 1
                if (early_step >= early_stopping_steps):
                    break
            
            print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}")
            print(f"SEED: {seed}, FOLD: {fold}, EPOCH: {epoch}, valid_loss: {valid_loss}, best_valid_loss: {best_loss}")

    #--------------------- PREDICTION ---------------------
    x_g_test_cat = np.concatenate((x_g_test, x_svd_g_test, x_cp_test), axis=1)
    x_c_test_cat = np.concatenate((x_c_test, x_svd_c_test, x_cp_test), axis=1)
    x_rg_g_test_cat = np.concatenate((x_rg_g_test, x_svd_rg_g_test, x_cp_test), axis=1)
    x_rg_c_test_cat = np.concatenate((x_rg_c_test, x_svd_rg_c_test, x_cp_test), axis=1)
    x_svd_g_test_cat = np.concatenate((x_svd_g_test, x_svd_rg_g_test), axis=1)
    x_svd_c_test_cat = np.concatenate((x_svd_c_test, x_svd_rg_c_test), axis=1)
    x_rg_g_rg_c_test_cat = np.concatenate((x_rg_g_test, x_rg_c_test, x_cp_test), axis=1)
    # x_g_c_test_cat = np.concatenate((x_g_test, x_c_test, x_cp_test), axis=1)
    # x_svd_g_svd_c_test_cat = np.concatenate((x_svd_g_test, x_svd_c_test), axis=1)
    # x_svd_rg_g_svd_rg_c_test_cat = np.concatenate((x_svd_rg_g_test, x_svd_rg_c_test), axis=1)

    testdataset = TestDataset(x_g_test_cat, x_c_test_cat, x_rg_g_test_cat, x_rg_c_test_cat,
                              x_svd_g_test_cat, x_svd_c_test_cat, x_rg_g_rg_c_test_cat)
    # testdataset = TestDataset(x_g_test_cat, x_c_test_cat,
    #                          x_rg_g_test_cat, x_rg_c_test_cat,
    #                          x_svd_g_test_cat, x_svd_c_test_cat,
    #                          x_g_c_test_cat, x_rg_g_rg_c_test_cat, x_svd_g_svd_c_test_cat, x_svd_rg_g_svd_rg_c_test_cat)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)

    model = Model(
        num_g_feats=num_gfeats + num_svd_gfeats + num_cpfeats,
        num_c_feats=num_cfeats + num_svd_cfeats + num_cpfeats,
        num_rg_g_feats=num_rg_gfeats + num_svd_rg_gfeats + num_cpfeats,
        num_rg_c_feats=num_rg_cfeats + num_svd_rg_cfeats + num_cpfeats,
        num_svd_g_feats=num_svd_gfeats + num_svd_rg_gfeats,
        num_svd_c_feats=num_svd_cfeats + num_svd_rg_cfeats,
        num_rg_g_rg_c_feats=num_rg_gfeats + num_rg_cfeats + num_cpfeats
        # num_g_c_feats=num_gfeats + num_cfeats + num_cpfeats,
        # num_svd_g_svd_c_feats=num_svd_gfeats + num_svd_cfeats,
        # num_svd_rg_g_svd_rg_c_feats=num_svd_rg_gfeats + num_svd_rg_cfeats
    )

    if INFERENCE == True:
        if model_type == 'normal':
            print(f"NORMAL_SEED{seed}_FOLD{fold}")
            model.load_state_dict(torch.load(f"{MODEL_PATH}/NORMAL_SEED{seed}_FOLD{fold}.pth"))
        if model_type == 'chris':
            print(f"CHRIS_SEED{seed}_FOLD{fold}")
            model.load_state_dict(torch.load(f"{MODEL_PATH}/CHRIS_SEED{seed}_FOLD{fold}.pth"))
        if model_type == 'fkubota':
            print(f"FKUBOTA_SEED{seed}_FOLD{fold}")
            model.load_state_dict(torch.load(f"{MODEL_PATH}/FKUBOTA_SEED{seed}_FOLD{fold}.pth"))
    if INFERENCE == False:
        if model_type == 'normal':
            model.load_state_dict(torch.load(f"NORMAL_SEED{seed}_FOLD{fold}.pth"))
        if model_type == 'chris':
            model.load_state_dict(torch.load(f"CHRIS_SEED{seed}_FOLD{fold}.pth"))
        if model_type == 'fkubota':
            model.load_state_dict(torch.load(f"FKUBOTA_SEED{seed}_FOLD{fold}.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

In [10]:
def calculate_metric(data, seed, n_splits):
    diff_mean = []
    Fold = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    for n, (train_index, val_index) in enumerate(Fold.split(data, data)):
        train_mean_positivity = data.iloc[train_index, 1:].mean()
        val_mean_positivity = data.iloc[val_index, 1:].mean()
        diff = train_mean_positivity - val_mean_positivity
        diff_mean += [np.sqrt(np.sum(diff**2.))]
    mn = np.mean(diff_mean)
    std = np.std(diff_mean)
    return i, mn, std

In [11]:
os.listdir('../input/lish-moa')

['sample_submission.csv',
 'train_drug.csv',
 'train_targets_scored.csv',
 'train_targets_nonscored.csv',
 'train_features.csv',
 'test_features.csv']

In [12]:
# Files Host provided
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
train_drug = pd.read_csv('../input/lish-moa/train_drug.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

# Custom files
# train_targets_nonscored_for_train = pd.read_csv('../input/moa-nonscored-features/train_nonscored_submission.csv')
# train_targets_nonscored_for_test = pd.read_csv('../input/moa-nonscored-features/test_nonscored_submission.csv')
group696 = pd.read_csv('../input/moa-nb004-696group/group.csv')

In [13]:
# train_features = train_features[train_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
# test_features = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

In [14]:
train = train_features.merge(train_targets_scored, on='sig_id', how='left')
train = train.reset_index(drop=True)
test = test_features.reset_index(drop=True)
target = train[train_targets_scored.columns]
TARGET_COLS = target.drop('sig_id', axis=1).columns.values.tolist()
NONSCORED_COLS = train_targets_nonscored.columns.values.tolist()
NONSCORED_COLS.remove('sig_id')

In [15]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 50
# BATCH_SIZE = 128
BATCH_SIZE = 256
# BATCH_SIZE = 512
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
# NFOLDS = 7
NFOLDS = 5
SEED = [0, 1, 2, 3, 4, 5]
EARLY_STOPPING_STEPS = 5
EARLY_STOP = False

MIXUP = True
MIXUP_ALPHA = 0.15
MIXUP_SEED_FOLD = np.zeros((len(SEED), NFOLDS))
# Specify SEED_No, Fold_No in MIXUP_SEED_FOLD[SEED,FOLD]
MIXUP_SEED_FOLD[0, 0] = 1
MIXUP_SEED_FOLD[0, 2] = 1
MIXUP_SEED_FOLD[1, 2] = 1
MIXUP_SEED_FOLD[2, 0] = 1
MIXUP_SEED_FOLD[2, 2] = 1
MIXUP_SEED_FOLD[2, 3] = 1
MIXUP_SEED_FOLD[3, 1] = 1
MIXUP_SEED_FOLD[3, 3] = 1
# MIXUP_SEED_FOLD[3, 4] = 1
MIXUP_SEED_FOLD[5, 1] = 1
MIXUP_SEED_FOLD[5, 2] = 1
MIXUP_SEED_FOLD[5, 4] = 1

LABEL_SMOOTHING = True
STANDARDSCALER = False

# Inference Only
INFERENCE = True
MODEL_PATH = '../input/moa-nn-models-chris-cv-with-nonscored'
# MODEL_PATH = '../input/moa-nn-models-normal-chris-blend'
# MODEL_PATH = '../input/moa-nn-models-chris-cv'

# CV scheme
NORMAL_CHRIS_BLEND = False
NORMAL_CV = False
CHRIS_CV = True
FKUBOTA_CV = False

train_org = train.copy()
target_org = target.copy()
train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test = test[test['cp_type']!='ctl_vehicle'].reset_index(drop=True)
target = train[train_targets_scored.columns]

# num_features=len(feature_cols)
NUM_TARGETS=len(TARGET_COLS)
HIDDEN_SIZE_CP = 16
HIDDEN_SIZE_G = 256
HIDDEN_SIZE_C = 256
HIDDEN_SIZE_RG_G = 256
HIDDEN_SIZE_RG_C = 256
HIDDEN_SIZE_SVD_G = 128
HIDDEN_SIZE_SVD_C = 128
HIDDEN_SIZE_RG_G_RG_C = 256
# HIDDEN_SIZE_G_C = 256
# HIDDEN_SIZE_SVD_G_SVD_C = 128
# HIDDEN_SIZE_SVD_RG_G_SVD_RG_C = 128
HIDDEN_SIZE = 1024

In [16]:
# Seed Selection
# https://www.kaggle.com/tolgadincer/iter-strat-some-seeds-are-better-than-others
# a = []
# train_targets_scored_seed = train_targets_scored.copy()
# train_targets_scored_seed = train_targets_scored_seed.set_index('sig_id')
# for i in range(100):
#    a += [calculate_metric(train_targets_scored_seed, i, 7)]
# seed_results = pd.DataFrame(a, columns=['Seed', 'mean', 'std']).sort_values(by='std')
# seed_results

In [17]:
# seed_list = seed_results['Seed'].values
# seed_list[:6]

In [18]:
def get_chris_cv_scheme_folds(n_splits, folds, scored, drug, seed):
    # MERGE DRUGS
    folds = folds.merge(drug, on='sig_id', how='left')
    scored = scored.merge(drug, on='sig_id', how='left')

    # LOCATE DRUGS
    vc = scored.drug_id.value_counts()
    vc1 = vc.loc[vc<=18].index.sort_values()
    vc2 = vc.loc[vc>18].index.sort_values()
    
    # STRATIFY DRUGS 18X OR LESS
    dct1 = {}; dct2 = {}
    skf = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    # tmp = scored.groupby('drug_id')[TARGET_COLS].mean().loc[vc1]
    # for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[TARGET_COLS])):
    tmp = scored.groupby('drug_id')[TARGET_COLS + NONSCORED_COLS].mean().loc[vc1]
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[TARGET_COLS + NONSCORED_COLS])):
        dd = {k:fold for k in tmp.index[idxV].values}
        dct1.update(dd)
        
    # STRATIFY DRUGS MORE THAN 18X
    skf = MultilabelStratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
    # for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[TARGET_COLS])):
    for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[TARGET_COLS + NONSCORED_COLS])):
        dd = {k:fold for k in tmp.sig_id[idxV].values}
        dct2.update(dd)
        
    # ASSIGN FOLDS
    scored['kfold'] = scored.drug_id.map(dct1)
    scored.loc[scored.kfold.isna(),'kfold'] = scored.loc[scored.kfold.isna(),'sig_id'].map(dct2)
    scored.kfold = scored.kfold.astype('int8')
    folds = folds.merge(scored[['sig_id', 'kfold']], on='sig_id', how='left')
    
    return folds

In [19]:
def get_not_drug_leak_folds(n_splits, train_features, train_drug, gruoup696):
    '''
    ::validation scheme::
    - MoAのパターンが各foldにできるだけ分配されている
    - 薬剤リークを防いでいる
    - 各foldの数が均等になるようになっている
    n_splits 個のfoldを作成する。
    ただし、cp_type = trt_cp と、top8にはfold=-1を割り振っている。
    696group のcsv: https://www.kaggle.com/fkubota/moa-nb004-696group
    ::example::
    train_features = pd.read_csv("train_features.csv")
    train_drug = pd.read_csv("train_drug.csv")
    group696 = pd.read_csv("MoA_nb004_696group/group.csv")
    df_fold = get_not_drug_leak_folds(5, train_features, train_drug, group696)
    '''
    # Top8に最終Fold値を割り当てるのでn_splits - 1としている
    # Defaultは、Top8にkfold=-1があたっている為
    n_splits -= 1
    TOP8_DRUG = ['87d714366', '9f80f3f77', '8b87a7a83', '5628cb3ee', 'd08af5d4b', '292ab2c28', 'd50f18348', 'd1b47f29d']
    mask_trt = (train_features['cp_type'] == 'trt_cp').values
    # mask_top8 を作成
    mask_top8 = []
    for drug_id in train_drug.drug_id.values:
        if drug_id in TOP8_DRUG:
            mask_top8.append(True)
        else:
            mask_top8.append(False)
    mask_top8 = np.array(mask_top8)
    # trt かつ top8 以外を抜き出す
    # group = 0 は要素数が多いので一番最後にやるようにする
    drug_groups = group696[mask_trt & ~mask_top8].group.values
    groups = np.sort(group696[mask_trt & ~mask_top8].group.unique())
    groups = groups[1:]
    groups = np.append(groups, 0)
    # 各グループにfoldを割り振る
    tile = []
    train_drug_trt = train_drug[mask_trt & ~mask_top8]
    train_drug_trt['kfold'] = -1
    for i_grp, grp in enumerate(groups):
        if i_grp == 0:
            tile = np.arange(1, n_splits+1).astype(int)
        mask_grp = drug_groups == grp
        drug_rank = train_drug[mask_trt & ~mask_top8][mask_grp].drug_id.value_counts()
        n_repeat = np.ceil(len(drug_rank)/n_splits).astype(int)
        folds = np.tile(tile, n_repeat)[:len(drug_rank)]
        for i, drug_id in enumerate(drug_rank.index.sort_values()):
            mask = train_drug_trt.drug_id.values == drug_id
            train_drug_trt.kfold[mask] = folds[i]
        tile = train_drug_trt.kfold.value_counts()[::-1][:n_splits].index
    train_drug_fold = train_drug.copy()
    # train_drug_fold['kfold'] = -1
    # train_drug_fold['kfold'][mask_trt & ~mask_top8] = train_drug_trt.kfold.values
    # Top8は1つのグループとして、Fold = n_splitの最終のFold値を設定する(例：n_split=5の場合、Top8のkfold = 4)
    # Fold値は0をスタートとする(なので、-1を引いている(train_drug_trt.kfold.values - 1))
    train_drug_fold['kfold'] = n_splits
    train_drug_fold['kfold'][mask_trt & ~mask_top8] = train_drug_trt.kfold.values - 1
    train_features = train_features.merge(train_drug_fold[['sig_id', 'kfold']], on='sig_id', how='left')

    # return train_drug_fold
    return train_features

In [20]:
# Averaging on multiple SEEDS
# Select Top 6 seed values with low_std
# SEED = seed_list[:6]
oof = np.zeros((len(train), len(TARGET_COLS)))
predictions = np.zeros((len(test), len(TARGET_COLS)))

test = test.drop('cp_type', axis=1)

for seed in SEED:
    if NORMAL_CV == True:
        normal_folds = train.copy()
        mskf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed)
        for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target)):
            normal_folds.loc[v_idx, 'kfold'] = int(f)
        normal_folds['kfold'] = normal_folds['kfold'].astype(int)
        normal_folds = normal_folds.drop(['cp_type'], axis=1)
        normal_oof_, normal_predictions_ = run_k_fold(normal_folds, NFOLDS, seed, 'normal')                                                          
    if CHRIS_CV == True:
        chris_folds = train.copy()
        scored = train_targets_scored.copy()
        nonscored = train_targets_nonscored.copy()
        scored_nonscored = scored.merge(nonscored, on='sig_id', how='left')
        drug = train_drug.copy()
        # chris_folds = get_chris_cv_scheme_folds(NFOLDS, chris_folds, scored, drug, seed)
        chris_folds = get_chris_cv_scheme_folds(NFOLDS, chris_folds, scored_nonscored, drug, seed)
        chris_folds = chris_folds.drop(['cp_type','drug_id'], axis=1)
        chris_oof_, chris_predictions_ = run_k_fold(chris_folds, NFOLDS, seed, 'chris')                                                          
    if FKUBOTA_CV == True:
        fkubota_folds = train_org.copy()        
        fkubota_folds = get_not_drug_leak_folds(NFOLDS, fkubota_folds, drug, group696)
        fkubota_folds = fkubota_folds[fkubota_folds['cp_type']!='ctl_vehicle'].reset_index(drop=True)
        fkubota_folds = fkubota_folds.drop('cp_type', axis=1)
        fkubota_oof_, fkubota_predictions_ = run_k_fold(fkubota_folds, NFOLDS, seed, 'fkubota')                                                          
    
    # oof_, predictions_ = run_k_fold(folds, NFOLDS, seed)
    if NORMAL_CHRIS_BLEND == True:
        oof += (normal_oof_ + chris_oof_ ) / (2 * len(SEED))
        # predictions += (0.5 * normal_predictions_ + 0.5 * chris_predictions_) / len(SEED)
        predictions += (0.3 * normal_predictions_ + 0.7 * chris_predictions_) / len(SEED)
    if NORMAL_CHRIS_BLEND == False:
        oof += chris_oof_ / len(SEED)
        predictions += chris_predictions_ / len(SEED)
    
train[TARGET_COLS] = oof
test[TARGET_COLS] = predictions

cpfeats : 5
gfeats : 761
cfeats : 100
rg_gfeats : 772
rg_cfeats : 100
svd_gfeats : 50
svd_cfeats : 31 
svd_rg_gfeats : 50
svd_rg_cfeats : 49
CHRIS_SEED0_FOLD0
cpfeats : 5
gfeats : 761
cfeats : 100
rg_gfeats : 772
rg_cfeats : 100
svd_gfeats : 50
svd_cfeats : 32 
svd_rg_gfeats : 50
svd_rg_cfeats : 50
CHRIS_SEED0_FOLD1
cpfeats : 5
gfeats : 762
cfeats : 100
rg_gfeats : 772
rg_cfeats : 100
svd_gfeats : 50
svd_cfeats : 33 
svd_rg_gfeats : 50
svd_rg_cfeats : 50
CHRIS_SEED0_FOLD2
cpfeats : 5
gfeats : 762
cfeats : 100
rg_gfeats : 772
rg_cfeats : 100
svd_gfeats : 50
svd_cfeats : 33 
svd_rg_gfeats : 50
svd_rg_cfeats : 49
CHRIS_SEED0_FOLD3
cpfeats : 5
gfeats : 762
cfeats : 100
rg_gfeats : 772
rg_cfeats : 100
svd_gfeats : 50
svd_cfeats : 33 
svd_rg_gfeats : 50
svd_rg_cfeats : 50
CHRIS_SEED0_FOLD4
cpfeats : 5
gfeats : 763
cfeats : 100
rg_gfeats : 772
rg_cfeats : 100
svd_gfeats : 50
svd_cfeats : 32 
svd_rg_gfeats : 50
svd_rg_cfeats : 50
CHRIS_SEED1_FOLD0
cpfeats : 5
gfeats : 762
cfeats : 100
rg_gfeat

In [21]:
# Calculate log_loss in Training Mode
if INFERENCE == False:
    valid_results = train_targets_scored.drop(columns=TARGET_COLS).merge(train[['sig_id']+TARGET_COLS], on='sig_id', how='left').fillna(0)
    
    y_true = train_targets_scored[TARGET_COLS].values
    y_pred = valid_results[TARGET_COLS].values
    
    score = 0
    for i in range(len(TARGET_COLS)):
        score_ = log_loss(y_true[:, i], y_pred[:, i])
        # score += score_ / target.shape[1]
        score += score_ / len(TARGET_COLS)
    print("CV log_loss: ", score)

In [22]:
# Output oof.csv in Training Mode
if INFERENCE == False:
    oof_results = train_targets_scored.drop(columns=TARGET_COLS).merge(train[['sig_id']+TARGET_COLS], on='sig_id', how='left').fillna(0)
    oof_results.to_csv('oof.csv', index=False)

In [23]:
# Make submission.csv
nikonekonn = sample_submission.drop(columns=TARGET_COLS).merge(test[['sig_id']+TARGET_COLS], on='sig_id', how='left').fillna(0)
nikonekonn.head(20)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [24]:
sub = nikonekonn.copy()

# cool_rabbit NN4

In [25]:
data_dir = '../input/lish-moa/'
os.listdir(data_dir)

['sample_submission.csv',
 'train_drug.csv',
 'train_targets_scored.csv',
 'train_targets_nonscored.csv',
 'train_features.csv',
 'test_features.csv']

In [26]:
train_features = pd.read_csv(data_dir + 'train_features.csv')
train_targets_scored = pd.read_csv(data_dir + 'train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(data_dir + 'train_targets_nonscored.csv')
train_drug = pd.read_csv(data_dir + 'train_drug.csv')
test_features = pd.read_csv(data_dir + 'test_features.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')

In [27]:
SEED_VALUE = 42

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=SEED_VALUE)

In [28]:
train_features2=train_features.copy()
test_features2=test_features.copy()

In [29]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [30]:
# Quantile Transformation

qt = QuantileTransformer(n_quantiles=100,random_state=42,output_distribution='normal')
data = pd.concat([pd.DataFrame(train_features[GENES+CELLS]), pd.DataFrame(test_features[GENES+CELLS])])
data2 = qt.fit_transform(data[GENES+CELLS])
train_features[GENES+CELLS] = pd.DataFrame(data2[:train_features.shape[0]])
test_features[GENES+CELLS] = pd.DataFrame(data2[-test_features.shape[0]:])

In [31]:
# GENES
n_comp = 600

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; 
test2 = data2[-test_features.shape[0]:]

train_gpca = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test_gpca = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

train_features = pd.concat((train_features, train_gpca), axis=1)
test_features = pd.concat((test_features, test_gpca), axis=1)

In [32]:
# CELLS
n_comp = 50

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train_cpca = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test_cpca = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

train_features = pd.concat((train_features, train_cpca), axis=1)
test_features = pd.concat((test_features, test_cpca), axis=1)

In [33]:
from sklearn.feature_selection import VarianceThreshold

var_thresh = VarianceThreshold(0.9)
data = train_features.append(test_features)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]

train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)

test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

In [34]:
from sklearn.cluster import KMeans
def fe_cluster(train, test, n_clusters_g=22, n_clusters_c=4, SEED=42):
    
    features_g = GENES
    features_c = CELLS
    
    def create_cluster(train, test, features, kind='g', n_clusters=n_clusters_g):
        train_ = train[features].copy()
        test_ = test[features].copy()
        data = pd.concat([train_, test_], axis=0)
        kmeans = KMeans(n_clusters=n_clusters, random_state=SEED).fit(data)
        train[f'clusters_{kind}'] = kmeans.labels_[:train.shape[0]]
        test[f'clusters_{kind}'] = kmeans.labels_[train.shape[0]:]
        train = pd.get_dummies(train, columns = [f'clusters_{kind}'])
        test = pd.get_dummies(test, columns = [f'clusters_{kind}'])
        return train, test
    
    train, test = create_cluster(train, test, features_g, kind = 'g', n_clusters = n_clusters_g)
    train, test = create_cluster(train, test, features_c, kind = 'c', n_clusters = n_clusters_c)
    return train, test

train_features2, test_features2 = fe_cluster(train_features2, test_features2)

In [35]:
train_pca=pd.concat((train_gpca,train_cpca),axis=1)
test_pca=pd.concat((test_gpca,test_cpca),axis=1)

In [36]:
def fe_cluster_pca(train, test, n_clusters=5, SEED=42):
    data=pd.concat([train, test], axis=0)
    kmeans = KMeans(n_clusters=n_clusters, random_state=SEED).fit(data)
    train[f'clusters_pca'] = kmeans.labels_[:train.shape[0]]
    test[f'clusters_pca'] = kmeans.labels_[train.shape[0]:]
    train = pd.get_dummies(train, columns = [f'clusters_pca'])
    test = pd.get_dummies(test, columns = [f'clusters_pca'])
    return train, test
train_cluster_pca, test_cluster_pca = fe_cluster_pca(train_pca, test_pca)

In [37]:
train_cluster_pca = train_cluster_pca.iloc[:, 650:]
test_cluster_pca = test_cluster_pca.iloc[:, 650:]

In [38]:
train_features_cluster = train_features2.iloc[:, 876:]
test_features_cluster = test_features2.iloc[:, 876:]

In [39]:
gsquarecols = ['g-574','g-211','g-216','g-0','g-255','g-577','g-153','g-389','g-60','g-370','g-248','g-167','g-203','g-177','g-301','g-332','g-517','g-6','g-744','g-224','g-162','g-3','g-736','g-486','g-283','g-22','g-359','g-361','g-440','g-335','g-106','g-307','g-745','g-146','g-416','g-298','g-666','g-91','g-17','g-549','g-145','g-157','g-768','g-568','g-396']

In [40]:
def fe_stats(train, test):
    
    features_g = GENES
    features_c = CELLS
    
    for df in train, test:
        df['g_sum'] = df[features_g].sum(axis = 1)
        df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1)
        df['g_kurt'] = df[features_g].kurtosis(axis = 1)
        df['g_skew'] = df[features_g].skew(axis = 1)
        df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1)
        df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        df['c_skew'] = df[features_c].skew(axis = 1)
        df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
        df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
        df['gc_std'] = df[features_g + features_c].std(axis = 1)
        df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
        df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        
        df['c52_c42'] = df['c-52'] * df['c-42']
        df['c13_c73'] = df['c-13'] * df['c-73']
        df['c26_c13'] = df['c-23'] * df['c-13']
        df['c33_c6'] = df['c-33'] * df['c-6']
        df['c11_c55'] = df['c-11'] * df['c-55']
        df['c38_c63'] = df['c-38'] * df['c-63']
        df['c38_c94'] = df['c-38'] * df['c-94']
        df['c13_c94'] = df['c-13'] * df['c-94']
        df['c4_c52'] = df['c-4'] * df['c-52']
        df['c4_c42'] = df['c-4'] * df['c-42']
        df['c13_c38'] = df['c-13'] * df['c-38']
        df['c55_c2'] = df['c-55'] * df['c-2']
        df['c55_c4'] = df['c-55'] * df['c-4']
        df['c4_c13'] = df['c-4'] * df['c-13']
        df['c82_c42'] = df['c-82'] * df['c-42']
        df['c66_c42'] = df['c-66'] * df['c-42']
        df['c6_c38'] = df['c-6'] * df['c-38']
        df['c2_c13'] = df['c-2'] * df['c-13']
        df['c62_c42'] = df['c-62'] * df['c-42']
        df['c90_c55'] = df['c-90'] * df['c-55']
        
        
        for feature in features_c:
             df[f'{feature}_squared'] = df[feature] ** 2     
                
        for feature in gsquarecols:
            df[f'{feature}_squared'] = df[feature] ** 2        
        
    return train, test

train_features2, test_features2 = fe_stats(train_features2, test_features2)

In [41]:
train_features_stats = train_features2.iloc[:, 902:]
test_features_stats = test_features2.iloc[:, 902:]

In [42]:
train_features = pd.concat((train_features, train_features_cluster,train_cluster_pca,train_features_stats), axis=1)
test_features = pd.concat((test_features, test_features_cluster,test_cluster_pca,test_features_stats), axis=1)

In [43]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored, on='sig_id')
train = train.merge(train_drug, on='sig_id')
train = train[train['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)
test = test_features[test_features['cp_type'] != 'ctl_vehicle'].reset_index(drop=True)
train = train.drop('cp_type', axis=1)
test = test.drop('cp_type', axis=1)

target_cols = [x for x in train_targets_scored.columns if x != 'sig_id']
aux_target_cols = [x for x in train_targets_nonscored.columns if x != 'sig_id']
all_target_cols = target_cols + aux_target_cols
num_targets = len(target_cols)
num_aux_targets = len(aux_target_cols)
num_all_targets = len(all_target_cols)

In [44]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)
        }
        
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }

        return dct

In [45]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y'].to(device)
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    return final_loss, valid_preds


def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    return preds

In [46]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1

        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
            
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [47]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets):
        super(Model, self).__init__()
        self.hidden_size = [1500, 1250, 1000, 750]
        self.dropout_value = [0.5, 0.35, 0.3, 0.25]

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.Linear(num_features, self.hidden_size[0])
        
        self.batch_norm2 = nn.BatchNorm1d(self.hidden_size[0])
        self.dropout2 = nn.Dropout(self.dropout_value[0])
        self.dense2 = nn.Linear(self.hidden_size[0], self.hidden_size[1])

        self.batch_norm3 = nn.BatchNorm1d(self.hidden_size[1])
        self.dropout3 = nn.Dropout(self.dropout_value[1])
        self.dense3 = nn.Linear(self.hidden_size[1], self.hidden_size[2])

        self.batch_norm4 = nn.BatchNorm1d(self.hidden_size[2])
        self.dropout4 = nn.Dropout(self.dropout_value[2])
        self.dense4 = nn.Linear(self.hidden_size[2], self.hidden_size[3])

        self.batch_norm5 = nn.BatchNorm1d(self.hidden_size[3])
        self.dropout5 = nn.Dropout(self.dropout_value[3])
        self.dense5 = nn.utils.weight_norm(nn.Linear(self.hidden_size[3], num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = F.leaky_relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.leaky_relu(self.dense3(x))

        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = F.leaky_relu(self.dense4(x))

        x = self.batch_norm5(x)
        x = self.dropout5(x)
        x = self.dense5(x)
        return x
    
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)

        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
            
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))  

In [48]:
class FineTuneScheduler:
    
    def __init__(self, epochs):
        self.epochs = epochs
        self.epochs_per_step = 0
        self.frozen_layers = []

    def copy_without_top(self, model, num_features, num_targets, num_targets_new):
        self.frozen_layers = []

        model_new = Model(num_features, num_targets)
        model_new.load_state_dict(model.state_dict())

        # Freeze all weights
        for name, param in model_new.named_parameters():
            layer_index = name.split('.')[0][-1]

            if layer_index == 5:
                continue

            param.requires_grad = False

            # Save frozen layer names
            if layer_index not in self.frozen_layers:
                self.frozen_layers.append(layer_index)

        self.epochs_per_step = self.epochs // len(self.frozen_layers)

        # Replace the top layers with another ones
        model_new.batch_norm5 = nn.BatchNorm1d(model_new.hidden_size[3])
        model_new.dropout5 = nn.Dropout(model_new.dropout_value[3])
        model_new.dense5 = nn.utils.weight_norm(nn.Linear(model_new.hidden_size[-1], num_targets_new))
        model_new.to(DEVICE)
        return model_new

    def step(self, epoch, model):
        if len(self.frozen_layers) == 0:
            return

        if epoch % self.epochs_per_step == 0:
            last_frozen_index = self.frozen_layers[-1]
            
            # Unfreeze parameters of the last frozen layer
            for name, param in model.named_parameters():
                layer_index = name.split('.')[0][-1]

                if layer_index == last_frozen_index:
                    param.requires_grad = True

            del self.frozen_layers[-1]  # Remove the last layer as unfrozen

In [49]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [50]:
feature_cols = [c for c in process_data(train).columns if c not in all_target_cols]
feature_cols = [c for c in feature_cols if c not in ['kfold', 'sig_id', 'drug_id']]
num_features = len(feature_cols)
num_features

1226

In [51]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 24
BATCH_SIZE = 128
WEIGHT_DECAY = {'ALL_TARGETS': 1e-5, 'SCORED_ONLY': 3e-6}
MAX_LR = {'ALL_TARGETS': 1e-2, 'SCORED_ONLY': 3e-3}
DIV_FACTOR = {'ALL_TARGETS': 1e3, 'SCORED_ONLY': 1e2}
PCT_START = 0.1

In [52]:
# Show model architecture
model = Model(num_features, num_all_targets)
model

Model(
  (batch_norm1): BatchNorm1d(1226, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dense1): Linear(in_features=1226, out_features=1500, bias=True)
  (batch_norm2): BatchNorm1d(1500, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (dense2): Linear(in_features=1500, out_features=1250, bias=True)
  (batch_norm3): BatchNorm1d(1250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout(p=0.35, inplace=False)
  (dense3): Linear(in_features=1250, out_features=1000, bias=True)
  (batch_norm4): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout4): Dropout(p=0.3, inplace=False)
  (dense4): Linear(in_features=1000, out_features=750, bias=True)
  (batch_norm5): BatchNorm1d(750, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout5): Dropout(p=0.25, inplace=False)
  (dense5): Linear(in_features=750, out_features=608, 

In [53]:
def make_cv_folds(train, SEEDS, NFOLDS, DRUG_THRESH):
    vc = train.drug_id.value_counts()
    vc1 = vc.loc[vc <= DRUG_THRESH].index.sort_values()
    vc2 = vc.loc[vc > DRUG_THRESH].index.sort_values()

    for seed_id in SEEDS:
        kfold_col = 'kfold_{}'.format(seed_id)
        
        # STRATIFY DRUGS 18X OR LESS
        dct1 = {}
        dct2 = {}

        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.groupby('drug_id')[target_cols].mean().loc[vc1]

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.index[idxV].values}
            dct1.update(dd)

        # STRATIFY DRUGS MORE THAN 18X
        skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=seed_id)
        tmp = train.loc[train.drug_id.isin(vc2)].reset_index(drop=True)

        for fold,(idxT, idxV) in enumerate(skf.split(tmp, tmp[target_cols])):
            dd = {k: fold for k in tmp.sig_id[idxV].values}
            dct2.update(dd)

        # ASSIGN FOLDS
        train[kfold_col] = train.drug_id.map(dct1)
        train.loc[train[kfold_col].isna(), kfold_col] = train.loc[train[kfold_col].isna(), 'sig_id'].map(dct2)
        train[kfold_col] = train[kfold_col].astype('int8')
        
    return train

SEEDS = [0, 1, 2, 3, 4, 5, 6]
NFOLDS = 7
DRUG_THRESH = 18

train = make_cv_folds(train, SEEDS, NFOLDS, DRUG_THRESH)
train.head()

sig_id cp_time cp_dose         0         1         2         3  \
0  id_000644bb2      24      D1  1.146806  0.902075 -0.418339 -0.961202   
1  id_000779bfc      72      D1  0.128824  0.676862  0.274345  0.090495   
2  id_000a6266a      48      D1  0.790372  0.939951  1.428097 -0.121817   
3  id_0015fd391      48      D1 -0.729866 -0.277163 -0.441200  0.766612   
4  id_001626bd3      72      D2 -0.444558 -0.481202  0.974729  0.977467   

          4         5         6         7         8         9        10  \
0 -0.254770 -1.021300 -1.369236 -0.029888  0.684319 -0.316668  1.526881   
1  1.208863  0.688965  0.316734  0.556428 -0.539718  0.831972 -1.255140   
2 -0.002067  1.495091  0.238763  0.363471 -0.003611  1.237966 -0.644480   
3  2.347817 -0.862761 -2.308829  0.305225 -0.191898 -1.389591 -1.005581   
4  1.468304 -0.874772 -0.372682 -0.212171 -1.067075  0.844018 -0.339762   

         11        12        13        14        15        16        17  \
0  0.180196  0.621873 -0.557740  0.282621 -1.060673 -1.139906  0.876619   
1 -0.573112 -0.208841  0.549210  0.149726  0.394322  0.402269  0.404046   
2 -0.768445 -0.127136 -2.323163  0.882413 -0.547770  0.473795 -0.402984   
3 -0.491974 -1.143377 -0.815236 -1.776327  1.392344 -0.225631 -1.054686   
4 -0.701653 -1.243135  1.187418  0.319514  0.344687 -0.023876 -1.570681   

         18        19        20        21        22        23        24  \
0  0.385452 -0.511010 -0.724484 -1.333353 -1.670424  1.247499  0.561705   
1 -0.837129  0.501094  1.241742 -0.134523 -0.390085 -0.444660  0.396115   
2 -0.308666 -0.040736 -0.109697 -1.008389 -1.938291  0.532560  0.626405   
3  0.248900 -2.125619  2.088503 -1.355700 -1.091855 -1.063889 -1.427285   
4 -0.846636 -0.705536 -0.447726 -0.293294  1.066411  0.492235  1.679156   

         25        26        27        28        29        30        31  \
0  0.406613  0.238226  0.161995 -0.521300  0.776699  0.419961 -1.097407   
1 -0.581092  0.624671  0.217307 -0.744494 -0.189269 -0.121230 -0.548652   
2  0.702705 -1.385532  2.391969 -0.096834  1.591787 -1.465673  0.815105   
3  0.004132 -1.057524  0.260284 -0.458109 -1.253958 -0.979995  0.126880   
4  1.005323  1.072014 -0.557841 -1.541019  0.373345 -0.154493  0.845777   

         32        33        34        35        36        37        38  \
0 -0.055064 -0.443270 -0.201534  0.276058  0.384696 -0.595063 -0.737635   
1  1.663916 -0.372043  0.352630 -0.284155  0.202425  0.779231 -0.354288   
2  0.737461  0.205613 -0.002542 -1.668236  0.147685 -0.161838  1.703524   
3 -2.099980  1.039492  0.781420 -2.137240 -1.418522 -1.066719  1.418550   
4  1.137909 -0.143263  1.489803 -1.022238 -1.330391  1.621226  0.775831   

         39        40        41        42        43        44        45  \
0 -0.371308 -0.006239 -0.339561 -1.362035  0.425598 -1.255837  0.330148   
1 -0.091953  0.211233 -0.521036 -1.344257  0.619376  1.862098  1.335946   
2  1.312477  0.553992 -0.721329  0.100235 -0.536122 -0.577234 -0.788790   
3 -0.639416 -0.884448 -0.000320  0.247921 -2.042377 -0.926970 -0.998886   
4 -0.155422  0.227374  0.169476  1.430737  0.280764  0.842108  1.390495   

         46        47        48        49        50        51        52  \
0 -0.343777 -0.022419  0.063678  0.700528 -0.186176 -0.658055  0.842025   
1  0.559412 -0.505327  0.238561 -1.095490  1.796571 -0.672133 -0.359793   
2  0.248539 -1.062270 -0.663437  0.743189 -0.154016 -0.347725  0.833083   
3 -0.413876  0.125047 -0.642957  1.026522 -1.328480 -0.807607 -1.440925   
4  1.482106 -1.280198 -0.491167 -0.998114 -0.129378 -0.508307  0.950318   

         53        54        55        56        57        58        59  \
0  0.567073 -0.148693 -0.228843 -0.769898  0.272742  0.801183  1.371216   
1 -1.004930 -0.772445 -0.174592 -0.732133  0.580915  0.440352 -0.855869   
2  0.322658 -1.088618 -0.603816  1.378014  0.056462 -0.261232 -1.288851   
3  1.714855 -0.935790 -1.360094 -0.691351  0.529634  0.094990 -1.325946   
4 -0.027241  0.155273  1.6

In [54]:
def run_training(fold_id, seed_id):
    seed_everything(seed_id)
    
    train_ = process_data(train)
    test_ = process_data(test)
    
    kfold_col = f'kfold_{seed_id}'
    trn_idx = train_[train_[kfold_col] != fold_id].index
    val_idx = train_[train_[kfold_col] == fold_id].index
    
    train_df = train_[train_[kfold_col] != fold_id].reset_index(drop=True)
    valid_df = train_[train_[kfold_col] == fold_id].reset_index(drop=True)
    
    def train_model(model, tag_name, target_cols_now, fine_tune_scheduler=None):
        x_train, y_train  = train_df[feature_cols].values, train_df[target_cols_now].values
        x_valid, y_valid =  valid_df[feature_cols].values, valid_df[target_cols_now].values
        
        train_dataset = MoADataset(x_train, y_train)
        valid_dataset = MoADataset(x_valid, y_valid)

        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
        
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=WEIGHT_DECAY[tag_name])
        scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,
                                                  steps_per_epoch=len(trainloader),
                                                  pct_start=PCT_START,
                                                  div_factor=DIV_FACTOR[tag_name], 
                                                  max_lr=MAX_LR[tag_name],
                                                  epochs=EPOCHS)
        
        loss_fn = nn.BCEWithLogitsLoss()
        loss_tr = SmoothBCEwLogits(smoothing=0.001)

        oof = np.zeros((len(train), len(target_cols_now)))
        best_loss = np.inf
        
        for epoch in range(EPOCHS):
            if fine_tune_scheduler is not None:
                fine_tune_scheduler.step(epoch, model)

            train_loss = train_fn(model, optimizer, scheduler, loss_tr, trainloader, DEVICE)
            valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
            print(f"SEED: {seed_id}, FOLD: {fold_id}, {tag_name}, EPOCH: {epoch}, train_loss: {train_loss:.6f}, valid_loss: {valid_loss:.6f}")

            if np.isnan(valid_loss):
                break
            
            if valid_loss < best_loss:
                best_loss = valid_loss
                oof[val_idx] = valid_preds
                torch.save(model.state_dict(), f"{tag_name}_Seed{seed_id}_FOLD{fold_id}_.pth")

        return oof

    fine_tune_scheduler = FineTuneScheduler(EPOCHS)

    pretrained_model = Model(num_features, num_all_targets)
    pretrained_model.to(DEVICE)

    # Train on scored + nonscored targets
    train_model(pretrained_model, 'ALL_TARGETS', all_target_cols)

    # Load the pretrained model with the best loss
    pretrained_model = Model(num_features, num_all_targets)
    pretrained_model.load_state_dict(torch.load(f"ALL_TARGETS_Seed{seed_id}_FOLD{fold_id}_.pth"))
    pretrained_model.to(DEVICE)

    # Copy model without the top layer
    final_model = fine_tune_scheduler.copy_without_top(pretrained_model, num_features, num_all_targets, num_targets)

    # Fine-tune the model on scored targets only
    oof = train_model(final_model, 'SCORED_ONLY', target_cols, fine_tune_scheduler)

    # Load the fine-tuned model with the best loss
    model = Model(num_features, num_targets)
    model.load_state_dict(torch.load(f"SCORED_ONLY_Seed{seed_id}_FOLD{fold_id}_.pth"))
    model.to(DEVICE)

    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    predictions = np.zeros((len(test_), num_targets))
    predictions = inference_fn(model, testloader, DEVICE)
    return oof, predictions

In [55]:
def run_k_fold(NFOLDS, seed_id):
    oof = np.zeros((len(train), len(target_cols)))
    predictions = np.zeros((len(test), len(target_cols)))
    
    for fold_id in range(NFOLDS):
        oof_, pred_ = run_training(fold_id, seed_id)
        oof += oof_
        predictions += pred_ / NFOLDS
        
    return oof, predictions

In [56]:
# Averaging on multiple SEEDS

SEED = [0, 1, 2, 3, 4, 5, 6]
oof = np.zeros((len(train), len(target_cols)))
predictions = np.zeros((len(test), len(target_cols)))

for seed_id in SEED:
    oof_, predictions_ = run_k_fold(NFOLDS, seed_id)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_cols] = oof
test[target_cols] = predictions

SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 0, train_loss: 0.494905, valid_loss: 0.018197
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 1, train_loss: 0.015655, valid_loss: 0.009692
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 2, train_loss: 0.013774, valid_loss: 0.009041
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 3, train_loss: 0.013025, valid_loss: 0.009121
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 4, train_loss: 0.014258, valid_loss: 0.009438
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 5, train_loss: 0.013330, valid_loss: 0.009365
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 6, train_loss: 0.013070, valid_loss: 0.009139
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 7, train_loss: 0.012952, valid_loss: 0.009047
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 8, train_loss: 0.012913, valid_loss: 0.009052
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 9, train_loss: 0.012913, valid_loss: 0.009060
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 10, train_loss: 0.012904, valid_loss: 0.009005
SEED: 0, FOLD: 0, ALL_TARGETS, EPOCH: 11, train_loss: 0.012838, valid_loss:

In [57]:
valid_results = train_targets_scored.drop(columns=target_cols).merge(train[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)

y_true = train_targets_scored[target_cols].values
y_pred = valid_results[target_cols].values

score = 0

for i in range(len(target_cols)):
    score += log_loss(y_true[:, i], y_pred[:, i])

print("CV log_loss: ", score / y_pred.shape[1])

CV log_loss:  0.015615037378683765


In [58]:
valid_results.to_csv('cr_nn4_oof.csv', index=False)

In [59]:
nn4 = sample_submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
nn4.head(20)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

# cool_rabbit TabNet

In [60]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl


In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler

import os
import gc
import random
import math
import time
from tqdm import tqdm_notebook as tqdm
import joblib
from pickle import load, dump 

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.modules.loss import _WeightedLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

import warnings
warnings.filterwarnings("ignore")
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [62]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
seed_everything(2020)

In [63]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
train_drug = pd.read_csv('../input/lish-moa/train_drug.csv')
train_group = pd.read_csv('../input/moa-nb004-696group/group.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [64]:
train_features = train_features[train_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test_features = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

In [65]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [66]:
# GENES

n_comp = 50
g = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
g_pca_ = load(open('../input/cool-rabbit-tabnet-fefiled/g_pca_.pkl', 'rb'))

g_pca = g_pca_.transform(g[GENES])
train_g_pca = g_pca[:train_features.shape[0]]
test_g_pca = g_pca[-test_features.shape[0]:]
train_g_pca_df = pd.DataFrame(train_g_pca, columns=[f'pca_G-{i}' for i in range(n_comp)])
test_g_pca_df = pd.DataFrame(test_g_pca, columns=[f'pca_G-{i}' for i in range(n_comp)])
train_features = pd.concat((train_features, train_g_pca_df), axis=1)
test_features = pd.concat((test_features, test_g_pca_df), axis=1)

In [67]:
# CELLS

n_comp = 25
c = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
c_pca_ = load(open('../input/cool-rabbit-tabnet-fefiled/c_pca_.pkl', 'rb'))

c_pca = c_pca_.transform(c[CELLS])
train_c_pca = c_pca[:train_features.shape[0]]
test_c_pca = c_pca[-test_features.shape[0]:]
train_c_pca_df = pd.DataFrame(train_c_pca, columns=[f'pca_C-{i}' for i in range(n_comp)])
test_c_pca_df = pd.DataFrame(test_c_pca, columns=[f'pca_C-{i}' for i in range(n_comp)])
train_features = pd.concat((train_features, train_c_pca_df), axis=1)
test_features = pd.concat((test_features, test_c_pca_df), axis=1)

In [68]:
# TruncatedSVD GENES 

n_comp = 50
g = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
tsvd = load(open('../input/cool-rabbit-tabnet-fefiled/g_tsvd.pkl', 'rb'))

train2 = tsvd.transform(train_features[GENES])
test2 = tsvd.transform(test_features[GENES])
train2 = pd.DataFrame(train2, columns=[f'SVD_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'SVD_G-{i}' for i in range(n_comp)])
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [69]:
# TruncatedSVD CELLS

n_comp = 25
c = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
tsvd = load(open('../input/cool-rabbit-tabnet-fefiled/c_tsvd.pkl', 'rb'))

train2 = tsvd.transform(train_features[CELLS])
test2 = tsvd.transform(test_features[CELLS])
train2 = pd.DataFrame(train2, columns=[f'SVD_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'SVD_C-{i}' for i in range(n_comp)])
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [70]:
# rankgauss GENES

RANKGAUSS_GENES = []
for col in GENES:
    RANKGAUSS_GENES.append(f'rankgauss_{col}')
    transformer = QuantileTransformer(n_quantiles=100, random_state=2020, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)
    train_features[f'rankgauss_{col}'] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[f'rankgauss_{col}'] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [71]:
# rankgauss CELLS

RANKGAUSS_CELLS = []
for col in CELLS:
    RANKGAUSS_CELLS.append(f'rankgauss_{col}')
    transformer = QuantileTransformer(n_quantiles=100, random_state=2020, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)
    train_features[f'rankgauss_{col}'] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[f'rankgauss_{col}'] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [72]:
def fe_stats(train, test):
    
    for df in train, test:
        df['g_sum'] = df[GENES].sum(axis=1)
        df['g_mean'] = df[GENES].mean(axis=1)
        df['g_std'] = df[GENES].std(axis=1)
        df['g_kurt'] = df[GENES].kurtosis(axis=1)
        df['g_skew'] = df[GENES].skew(axis=1)
        df['g_max'] = df[GENES].max(axis=1)
        df['g_min'] = df[GENES].min(axis=1)
        df['c_sum'] = df[CELLS].sum(axis=1)
        df['c_mean'] = df[CELLS].mean(axis=1)
        df['c_std'] = df[CELLS].std(axis=1)
        df['c_kurt'] = df[CELLS].kurtosis(axis=1)
        df['c_skew'] = df[CELLS].skew(axis=1)
        df['c_max'] = df[CELLS].max(axis=1)
        df['c_min'] = df[CELLS].min(axis=1)
        df['gc_sum'] = df[GENES+CELLS].sum(axis=1)
        df['gc_mean'] = df[GENES+CELLS].mean(axis=1)
        df['gc_std'] = df[GENES+CELLS].std(axis=1)
        df['gc_kurt'] = df[GENES+CELLS].kurtosis(axis=1)
        df['gc_skew'] = df[GENES+CELLS].skew(axis=1)
        
    return train, test

train_features, test_features = fe_stats(train_features, test_features)

In [73]:
def cate2num(df):
    df['cp_time'] = df['cp_time'].map({24: 0, 48: 1, 72: 2})
    df['cp_dose'] = df['cp_dose'].map({'D1': 0, 'D2': 1})
    return df

train_features = cate2num(train_features)
test_features = cate2num(test_features)

In [74]:
from sklearn.feature_selection import VarianceThreshold

var_thresh = VarianceThreshold(0.5)
data = train_features.append(test_features)
data_transformed_ = load(open('../input/cool-rabbit-tabnet-fefiled/data_transformed_.pkl', 'rb'))

data_transformed = data_transformed_.transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[:train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0]:]

train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4), columns=['sig_id','cp_type','cp_time','cp_dose'])
train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)
test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4), columns=['sig_id','cp_type','cp_time','cp_dose'])
test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.iloc[:, 2:] = train_features.iloc[:, 2:].astype('float64')
test_features.iloc[:, 2:] = test_features.iloc[:, 2:].astype('float64')

In [75]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_drug, on='sig_id')
train = train.drop('cp_type', axis=1)
test = test_features.drop('cp_type', axis=1)
target_cols = list(train_targets_scored.columns[1:])
target = train[target_cols].values

In [76]:
NFOLD = 10

In [77]:
vc = train.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()
# vc1 = vc.loc[(vc==6)|(vc==12)|(vc==18)].index.sort_values()
# vc2 = vc.loc[(vc!=6)&(vc!=12)&(vc!=18)].index.sort_values()
dct1 = {}
dct2 = {}

mlskf1 = MultilabelStratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=2020)
tmp1 = train.groupby('drug_id')[target_cols].mean().loc[vc1]
for fold1, (idxT1, idxV1) in enumerate(mlskf1.split(tmp1, tmp1[target_cols])):
    dd1 = {k: fold1 for k in tmp1.index[idxV1].values}
    dct1.update(dd1)
    
mlskf2 = MultilabelStratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=2020)
tmp2 = train.loc[train.drug_id.isin(vc2)].reset_index(drop=True)
for fold2, (idxT2, idxV2) in enumerate(mlskf2.split(tmp2, tmp2[target_cols])):
    dd2 = {k: fold2 for k in tmp2.sig_id[idxV2].values}
    dct2.update(dd2)

# ASSIGN FOLDS
train['fold'] = train.drug_id.map(dct1)
train.loc[train.fold.isna(),'fold'] = train.loc[train.fold.isna(),'sig_id'].map(dct2)
train.fold = train.fold.astype('int8')

In [78]:
folds = train.copy()
feature_cols = [c for c in folds.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['fold', 'sig_id', 'drug_id']]
len(feature_cols)

1898

In [79]:
class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1 - y_true) * np.log(1 - logits + 1e-15) + y_true * np.log(logits + 1e-15)
        return np.mean(-aux)

In [80]:
predictions = np.zeros((len(test), len(target_cols)))
X_test = test.iloc[:, 1:].values
SEED = [2020, 1, 777, 28, 42, 123]
for seed in SEED:
    print(f'===============seed: {seed}===============')
    seed_everything(seed)
    
    for fold in range(NFOLD):
        joblib_file = f"../input/cool-rabbit-tabnet-fefiled/seed_{seed}_fold_{fold}.pth"
        model = joblib.load(joblib_file)
        preds_test = model.predict(X_test)
        preds_test_ = 1 / (1 + np.exp(-preds_test))
        predictions += (preds_test_ / (NFOLD * len(SEED)))

===============seed: 2020===============
===============seed: 1===============
===============seed: 777===============
===============seed: 28===============
===============seed: 42===============
===============seed: 123===============


In [81]:
test[target_cols] = predictions

In [82]:
tabnet = submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
tabnet.head(20)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

# cool_rabbit NN1 normal

In [83]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(2020)

In [84]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
train_drug = pd.read_csv('../input/lish-moa/train_drug.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [85]:
train_features = train_features[train_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)
test_features = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

In [86]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [87]:
# GENES

n_comp = 50
g = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
g_pca_ = load(open('../input/cool-rabbit-nn-fefiled/g_pca_.pkl', 'rb'))

g_pca = g_pca_.transform(g[GENES])
train_g_pca = g_pca[:train_features.shape[0]]
test_g_pca = g_pca[-test_features.shape[0]:]
train_g_pca_df = pd.DataFrame(train_g_pca, columns=[f'pca_G-{i}' for i in range(n_comp)])
test_g_pca_df = pd.DataFrame(test_g_pca, columns=[f'pca_G-{i}' for i in range(n_comp)])
train_features = pd.concat((train_features, train_g_pca_df), axis=1)
test_features = pd.concat((test_features, test_g_pca_df), axis=1)

In [88]:
# CELLS

n_comp = 25
c = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
c_pca_ = load(open('../input/cool-rabbit-nn-fefiled/c_pca_.pkl', 'rb'))

c_pca = c_pca_.transform(c[CELLS])
train_c_pca = c_pca[:train_features.shape[0]]
test_c_pca = c_pca[-test_features.shape[0]:]
train_c_pca_df = pd.DataFrame(train_c_pca, columns=[f'pca_C-{i}' for i in range(n_comp)])
test_c_pca_df = pd.DataFrame(test_c_pca, columns=[f'pca_C-{i}' for i in range(n_comp)])
train_features = pd.concat((train_features, train_c_pca_df), axis=1)
test_features = pd.concat((test_features, test_c_pca_df), axis=1)

In [89]:
# rankgauss GENES

RANKGAUSS_GENES = []
for col in GENES:
    RANKGAUSS_GENES.append(f'rankgauss_{col}')
    transformer = QuantileTransformer(n_quantiles=100, random_state=2020, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)
    train_features[f'rankgauss_{col}'] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[f'rankgauss_{col}'] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [90]:
# rankgauss CELLS

RANKGAUSS_CELLS = []
for col in CELLS:
    RANKGAUSS_CELLS.append(f'rankgauss_{col}')
    transformer = QuantileTransformer(n_quantiles=100, random_state=2020, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)
    train_features[f'rankgauss_{col}'] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[f'rankgauss_{col}'] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [91]:
# TruncatedSVD GENES 

n_comp = 50
tsvd = TruncatedSVD(n_components=n_comp, n_iter=10, random_state=2020).fit(train_features[GENES])
train2 = tsvd.transform(train_features[GENES])
test2 = tsvd.transform(test_features[GENES])
train2 = pd.DataFrame(train2, columns=[f'SVD_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'SVD_G-{i}' for i in range(n_comp)])
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [92]:
# TruncatedSVD CELLS

n_comp = 25
tsvd = TruncatedSVD(n_components=n_comp, n_iter=10, random_state=2020).fit(train_features[CELLS])
train2 = tsvd.transform(train_features[CELLS])
test2 = tsvd.transform(test_features[CELLS])
train2 = pd.DataFrame(train2, columns=[f'SVD_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'SVD_C-{i}' for i in range(n_comp)])
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [93]:
def fe_stats(train, test):
    
    for df in train, test:
        df['g_sum'] = df[GENES].sum(axis=1)
        df['g_mean'] = df[GENES].mean(axis=1)
        df['g_std'] = df[GENES].std(axis=1)
        df['g_kurt'] = df[GENES].kurtosis(axis=1)
        df['g_skew'] = df[GENES].skew(axis=1)
        df['g_max'] = df[GENES].max(axis=1)
        df['g_min'] = df[GENES].min(axis=1)
        df['c_sum'] = df[CELLS].sum(axis=1)
        df['c_mean'] = df[CELLS].mean(axis=1)
        df['c_std'] = df[CELLS].std(axis=1)
        df['c_kurt'] = df[CELLS].kurtosis(axis=1)
        df['c_skew'] = df[CELLS].skew(axis=1)
        df['c_max'] = df[CELLS].max(axis=1)
        df['c_min'] = df[CELLS].min(axis=1)
        df['gc_sum'] = df[GENES+CELLS].sum(axis=1)
        df['gc_mean'] = df[GENES+CELLS].mean(axis=1)
        df['gc_std'] = df[GENES+CELLS].std(axis=1)
        df['gc_kurt'] = df[GENES+CELLS].kurtosis(axis=1)
        df['gc_skew'] = df[GENES+CELLS].skew(axis=1)
        
    return train, test

train_features, test_features = fe_stats(train_features, test_features)

In [94]:
def cate2num(df):
    df['cp_time'] = df['cp_time'].map({24: 0, 48: 1, 72: 2})
    df['cp_dose'] = df['cp_dose'].map({'D1': 0, 'D2': 1})
    return df

train_features = cate2num(train_features)
test_features = cate2num(test_features)

In [95]:
from sklearn.feature_selection import VarianceThreshold

var_thresh = VarianceThreshold(0.5)
data = train_features.append(test_features)
data_transformed_ = load(open('../input/cool-rabbit-nn-fefiled/data_transformed_.pkl', 'rb'))

data_transformed = data_transformed_.transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[:train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0]:]

train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4), columns=['sig_id','cp_type','cp_time','cp_dose'])
train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)
test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4), columns=['sig_id','cp_type','cp_time','cp_dose'])
test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.iloc[:, 2:] = train_features.iloc[:, 2:].astype('float64')
test_features.iloc[:, 2:] = test_features.iloc[:, 2:].astype('float64')

In [96]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_drug, on='sig_id')
train = train.drop('cp_type', axis=1)
test = test_features.drop('cp_type', axis=1)
target_cols = list(train_targets_scored.columns[1:])
target = train[target_cols].values

In [97]:
NFOLD = 10

In [98]:
vc = train.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()
# vc1 = vc.loc[(vc==6)|(vc==12)|(vc==18)].index.sort_values()
# vc2 = vc.loc[(vc!=6)&(vc!=12)&(vc!=18)].index.sort_values()
dct1 = {}
dct2 = {}

mlskf1 = MultilabelStratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=2020)
tmp1 = train.groupby('drug_id')[target_cols].mean().loc[vc1]
for fold1, (idxT1, idxV1) in enumerate(mlskf1.split(tmp1, tmp1[target_cols])):
    dd1 = {k: fold1 for k in tmp1.index[idxV1].values}
    dct1.update(dd1)
    
mlskf2 = MultilabelStratifiedKFold(n_splits=NFOLD, shuffle=True, random_state=2020)
tmp2 = train.loc[train.drug_id.isin(vc2)].reset_index(drop=True)
for fold2, (idxT2, idxV2) in enumerate(mlskf2.split(tmp2, tmp2[target_cols])):
    dd2 = {k: fold2 for k in tmp2.sig_id[idxV2].values}
    dct2.update(dd2)

# ASSIGN FOLDS
train['fold'] = train.drug_id.map(dct1)
train.loc[train.fold.isna(),'fold'] = train.loc[train.fold.isna(),'sig_id'].map(dct2)
train.fold = train.fold.astype('int8')

In [99]:
folds = train.copy()
feature_cols = [c for c in folds.columns if c not in target_cols]
feature_cols = [c for c in feature_cols if c not in ['fold', 'sig_id', 'drug_id']]
len(feature_cols)

1898

In [100]:
class TestDataset(Dataset):
    
    def __init__(self, df, features):
        self.data = df[features].values
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.FloatTensor(self.data[idx])
        return x

In [101]:
class SmoothBCEwLogits(_WeightedLoss):
    
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [102]:
class Model(nn.Module):
        
    def __init__(self, cfg):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(len(cfg.features))
        self.dropout1 = nn.Dropout(cfg.dropout)
        self.dense1 = nn.utils.weight_norm(nn.Linear(len(cfg.features), cfg.hidden_size))
        self.batch_norm2 = nn.BatchNorm1d(cfg.hidden_size)
        self.dropout2 = nn.Dropout(cfg.dropout)
        self.dense2 = nn.utils.weight_norm(nn.Linear(cfg.hidden_size, cfg.hidden_size))
        self.batch_norm3 = nn.BatchNorm1d(cfg.hidden_size)
        self.dropout3 = nn.Dropout(cfg.dropout)
        self.dense3 = nn.utils.weight_norm(nn.Linear(cfg.hidden_size, len(cfg.target_cols)))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x))
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.leaky_relu(self.dense2(x))
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        return x

In [103]:
def inference_fn(test_loader, model, device):
    model.eval()
    preds = []

    for step, x in enumerate(test_loader):
        x = x.to(device)
        
        with torch.no_grad():
            pred = model(x)
        preds.append(pred.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds


class AverageMeter(object):
    
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [104]:
def run_single_nn(cfg, train, test, folds, features, target, device, fold_num, seed):
    
    seed_everything(seed=seed) 
    test_dataset = TestDataset(test, features)
    test_loader = DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=4, pin_memory=True)
    model = Model(cfg)
    model.load_state_dict(torch.load(f"../input/cool-rabbit-nn-fefiled/fold{fold_num}_seed{seed}.pth"))
    model.to(device)
    predictions = inference_fn(test_loader, model, device)
    torch.cuda.empty_cache()

    return predictions


def run_kfold_nn(cfg, train, test, folds, features, target, device, n_fold, seed):
    
    predictions = np.zeros((len(test), len(cfg.target_cols)))

    for _fold in range(n_fold):
        _predictions = run_single_nn(cfg, train, test, folds, features, target, device, fold_num=_fold, seed=seed)
        predictions += _predictions / n_fold
    
    return predictions

In [105]:
class CFG:
    max_grad_norm=1000
    gradient_accumulation_steps=1
    hidden_size=500
    lr=1e-2
    dropout=0.5
    weight_decay=1e-6
    batch_size=256
    epochs=50
    early_stopping_steps=10
    EARLY_STOP=False
    features=feature_cols
    target_cols=target_cols

In [106]:
predictions = np.zeros((len(test), len(CFG.target_cols)))

SEED = [2020, 1, 777, 28, 42, 123]
for seed in SEED:
    print(f'===============seed: {seed}===============')
    _predictions = run_kfold_nn(CFG, train, test, folds, feature_cols, target, device, n_fold=NFOLD, seed=seed)
    predictions += _predictions / len(SEED)

===============seed: 2020===============
===============seed: 1===============
===============seed: 777===============
===============seed: 28===============
===============seed: 42===============
===============seed: 123===============


In [107]:
test[target_cols] = predictions

In [108]:
nn1n = submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
nn1n.head(20)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

In [109]:
nn1 = nn1n.copy()

# cool_rabbit NN1 mixup

In [110]:
def run_single_nn(cfg, train, test, folds, features, target, device, fold_num, seed):
    
    seed_everything(seed=seed)
    test_dataset = TestDataset(test, features)
    test_loader = DataLoader(test_dataset, batch_size=cfg.batch_size, shuffle=False, num_workers=4, pin_memory=True)
    model = Model(cfg)
    model.load_state_dict(torch.load(f"../input/cool-rabbit-nn1-fefiled-mixup/fold{fold_num}_seed{seed}.pth"))
    model.to(device)
    predictions = inference_fn(test_loader, model, device)
    torch.cuda.empty_cache()

    return predictions


def run_kfold_nn(cfg, train, test, folds, features, target, device, n_fold, seed):

    predictions = np.zeros((len(test), len(cfg.target_cols)))

    for _fold in range(n_fold):
        _predictions = run_single_nn(cfg, train, test, folds, features, target, device, fold_num=_fold, seed=seed)
        predictions += _predictions / n_fold
    
    return predictions

In [111]:
class CFG:
    max_grad_norm=1000
    gradient_accumulation_steps=1
    hidden_size=500
    lr=1e-2
    dropout=0.5
    weight_decay=1e-6
    batch_size=256
    epochs=50
    early_stopping_steps=10
    EARLY_STOP=False
    features=feature_cols
    target_cols=target_cols

In [112]:
predictions = np.zeros((len(test), len(CFG.target_cols)))

SEED = [2020, 1, 777, 28, 42, 123]
for seed in SEED:
    print(f'===============seed: {seed}===============')
    _predictions = run_kfold_nn(CFG, train, test, folds, feature_cols, target, device, n_fold=NFOLD, seed=seed)
    predictions += _predictions / len(SEED)

===============seed: 2020===============
===============seed: 1===============
===============seed: 777===============
===============seed: 28===============
===============seed: 42===============
===============seed: 123===============


In [113]:
test[target_cols] = predictions

In [114]:
nn1m = submission.drop(columns=target_cols).merge(test[['sig_id']+target_cols], on='sig_id', how='left').fillna(0)
nn1m.head(20)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib

# Processing of NN1

In [115]:
over200 = ['nfkb_inhibitor', 'proteasome_inhibitor', 'cyclooxygenase_inhibitor',
           'dopamine_receptor_antagonist', 'serotonin_receptor_antagonist',
           'dna_inhibitor', 'glutamate_receptor_antagonist',
           'adrenergic_receptor_antagonist', 'cdk_inhibitor', 'egfr_inhibitor',
           'tubulin_inhibitor', 'acetylcholine_receptor_antagonist',
           'pdgfr_inhibitor', 'hmgcr_inhibitor', 'calcium_channel_blocker',
           'flt3_inhibitor', 'kit_inhibitor', 'adrenergic_receptor_agonist',
           'sodium_channel_inhibitor', 'glucocorticoid_receptor_agonist',
           'phosphodiesterase_inhibitor', 'histamine_receptor_antagonist',
           'serotonin_receptor_agonist', 'raf_inhibitor']

In [116]:
under200 = [col for col in target_cols if col not in over200]

In [117]:
nn1[over200] = nn1n[over200]
nn1[under200] = nn1m[under200]

# Ensemble

In [118]:
sub[target_cols] = (nikonekonn[target_cols] * 0.6 + nn4[target_cols] * 0.4) * 0.875 + (tabnet[target_cols] * 0.45 + nn1[target_cols] * 0.55) * 0.125


In [119]:
sub.to_csv('submission.csv', index=False)
sub.head(20)

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,adrenergic_receptor_antagonist,akt_inhibitor,aldehyde_dehydrogenase_inhibitor,alk_inhibitor,ampk_activator,analgesic,androgen_receptor_agonist,androgen_receptor_antagonist,anesthetic_-_local,angiogenesis_inhibitor,angiotensin_receptor_antagonist,anti-inflammatory,antiarrhythmic,antibiotic,anticonvulsant,antifungal,antihistamine,antimalarial,antioxidant,antiprotozoal,antiviral,apoptosis_stimulant,aromatase_inhibitor,atm_kinase_inhibitor,atp-sensitive_potassium_channel_antagonist,atp_synthase_inhibitor,atpase_inhibitor,atr_kinase_inhibitor,aurora_kinase_inhibitor,autotaxin_inhibitor,bacterial_30s_ribosomal_subunit_inhibitor,bacterial_50s_ribosomal_subunit_inhibitor,bacterial_antifolate,bacterial_cell_wall_synthesis_inhibitor,bacterial_dna_gyrase_inhibitor,bacterial_dna_inhibitor,bacterial_membrane_integrity_inhibitor,bcl_inhibitor,bcr-abl_inhibitor,benzodiazepine_receptor_agonist,beta_amyloid_inhibitor,bromodomain_inhibitor,btk_inhibitor,calcineurin_inhibitor,calcium_channel_blocker,cannabinoid_receptor_agonist,cannabinoid_receptor_antagonist,carbonic_anhydrase_inhibitor,casein_kinase_inhibitor,caspase_activator,catechol_o_methyltransferase_inhibitor,cc_chemokine_receptor_antagonist,cck_receptor_antagonist,cdk_inhibitor,chelating_agent,chk_inhibitor,chloride_channel_blocker,cholesterol_inhibitor,cholinergic_receptor_antagonist,coagulation_factor_inhibitor,corticosteroid_agonist,cyclooxygenase_inhibitor,cytochrome_p450_inhibitor,dihydrofolate_reductase_inhibitor,dipeptidyl_peptidase_inhibitor,diuretic,dna_alkylating_agent,dna_inhibitor,dopamine_receptor_agonist,dopamine_receptor_antagonist,egfr_inhibitor,elastase_inhibitor,erbb2_inhibitor,estrogen_receptor_agonist,estrogen_receptor_antagonist,faah_inhibitor,farnesyltransferase_inhibitor,fatty_acid_receptor_agonist,fgfr_inhibitor,flt3_inhibitor,focal_adhesion_kinase_inhibitor,free_radical_scavenger,fungal_squalene_epoxidase_inhibitor,gaba_receptor_agonist,gaba_receptor_antagonist,gamma_secretase_inhibitor,glucocorticoid_receptor_agonist,glutamate_inhibitor,glutamate_receptor_agonist,glutamate_receptor_antagonist,gonadotropin_receptor_agonist,gsk_inhibitor,hcv_inhibitor,hdac_inhibitor,histamine_receptor_agonist,histamine_receptor_antagonist,histone_lysine_demethylase_inhibitor,histone_lysine_methyltransferase_inhibitor,hiv_inhibitor,hmgcr_inhibitor,hsp_inhibitor,igf-1_inhibitor,ikk_inhibitor,imidazoline_receptor_agonist,immunosuppressant,insulin_secretagogue,insulin_sensitizer,integrin_inhibitor,jak_inhibitor,kit_inhibitor,laxative,leukotriene_inhibitor,leukotriene_receptor_antagonist,lipase_inhibitor,lipoxygenase_inhibitor,lxr_agonist,mdm_inhibitor,mek_inhibitor,membrane_integrity_inhibitor,mineralocorticoid_receptor_antagonist,monoacylglycerol_lipase_inhibitor,monoamine_oxidase_inhibitor,monopolar_spindle_1_kinase_inhibitor,mtor_inhibitor,mucolytic_agent,neuropeptide_receptor_antagonist,nfkb_inhibitor,nicotinic_receptor_agonist,nitric_oxide_donor,nitric_oxide_production_inhibitor,nitric_oxide_synthase_inhibitor,norepinephrine_reuptake_inhibitor,nrf2_activator,opioid_receptor_agonist,opioid_receptor_antagonist,orexin_receptor_antagonist,p38_mapk_inhibitor,p-glycoprotein_inhibitor,parp_inhibitor,pdgfr_inhibitor,pdk_inhibitor,phosphodiesterase_inhibitor,phospholipase_inhibitor,pi3k_inhibitor,pkc_inhibitor,potassium_channel_activator,potassium_channel_antagonist,ppar_receptor_agonist,ppar_receptor_antagonist,progesterone_receptor_agonist,progesterone_receptor_antagonist,prostaglandin_inhibitor,prostanoid_receptor_antagonist,proteasome_inhibitor,protein_kinase_inhibitor,protein_phosphatase_inhibitor,protein_synthesis_inhibitor,protein_tyrosine_kinase_inhibitor,radiopaque_medium,raf_inhibitor,ras_gtpase_inhib